In [190]:
import urllib3
from urllib3.exceptions import InsecureRequestWarning

import os
import certifi
import pandas as pd
import requests

from functools import reduce

In [191]:
# 경고 끄기
urllib3.disable_warnings(InsecureRequestWarning)   

startYYYYMM = '201701'
endYYYYMM = '202212'

startYYYYQQ = '201701'
endYYYYQQ = '202212'

startYYYY = '2017'
endYYYY = '2022'

# 실행 시 kosisAPI 개인 인증키 사용하실 것
apiKey = 'NjgwMjNjNTc0NTBlZWM5Y2JjMmQ0YWEyNTIzMjhhNmM'

In [192]:
# URL request 함수
def urlRequest(url):
    r = requests.get(url, verify=False)
    jo = r.json()
    return  pd.DataFrame(jo)

In [193]:
# initQuaterDf 함수
# 데이터마다 시계열이 다르므로, 공통 시계열(2019Q1 ~ 2022Q4) DataFrame을 만들어서 추출된 데이터를 붙임(Left Join) 
# 연도 및 분기 리스트
def initQuaterDf():
    years = range(2018, 2023)  # 2018 ~ 2022
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']
    # PRD_DE 값 생성
    prd_de_list = [f"{year}{q}" for year in years for q in quarters]
    # DataFrame 생성 (열 이름이 'PRD_DE')
    return pd.DataFrame({'PRD_DE': prd_de_list})

In [194]:
# 추출 데이터가 월단위 데이터이므로, 이를 Quater 단위로 변환
# 월 → 분기(Q1~Q4)로 변환하는 함수 : 3,6,9,12월 자료만 Quater로 변환하고 나머지는 Unknown 처리
def month_to_quarter(prd):
    year = prd[:4]
    month = prd[-2:]
    if month in ['03']:
        quarter = 'Q1'
    elif month in ['06']:
        quarter = 'Q2'
    elif month in ['09']:
        quarter = 'Q3'
    elif month in ['12']:
        quarter = 'Q4'
    else:
        quarter = 'Unknown'
    return f"{year}{quarter}"

In [195]:
def save_excel_and_csv(df, filename):
    folder_name = 'kosis_output_files'
    # 1. 폴더가 없으면 생성
    os.makedirs(folder_name, exist_ok=True)
    # 2. 저장할 전체 경로 만들기
    excel_path = os.path.join(folder_name, f"{filename}.xlsx")
    csv_path = os.path.join(folder_name, f"{filename}.csv")
     # 3. 파일 저장
    df.to_excel(excel_path, index=False)
    df.to_csv(csv_path, index=False, encoding='utf-8')

In [196]:
# 소상공인 경기동향(BSI) 지수 (소상공인, 부문별 실적 및 전망)

# dataRefine 함수
def dataRefine(refDf):
    tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # API Data를 DataFrame형태로 저장할 딕셔너리
    split_dfs = {}
    # Row로 구분되어 있는 조건들(예, '체감', '전망')을 개개의 컬럼으로 만들어주기 위한 준비
    filter_1 = tempDf['ITM_NM'].drop_duplicates().values.tolist()
    filter_2 = tempDf['C1_NM'].drop_duplicates().values.tolist()
    # 조건 조합 반복
    for s in filter_1:
        for p in filter_2:
            # 'DT'컬럼의 이름을 대체함
            newColName = f"{tempDf['TBL_NM'].unique()[0][:]}_{p}_{s}"  
            filtered = tempDf[(tempDf['ITM_NM'] == s) & (tempDf['C1_NM'] == p)]
            split_dfs[newColName] = filtered.copy()
            split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
            split_dfs[newColName]['PRD_DE'] = split_dfs[newColName]['PRD_DE'].apply(month_to_quarter)
            # Unknown이 들어가있는 행을 삭제하여 Quater 데이터만 남김
            split_dfs[newColName] = split_dfs[newColName][~split_dfs[newColName]['PRD_DE'].str.contains('Unknown')]
    return split_dfs
 
def mergeTimeSeriesDataFrame(split_dfs):
    df = initQuaterDf()
    df_merged = df.copy()
    for name, sub in split_dfs.items():
        sub_unique = sub.drop_duplicates(subset=['PRD_DE'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 4]], on='PRD_DE', how='left', suffixes=('', '_dup'))  # 필요에 따라 'outer' 등으로 변경
    return df_merged
    

##################### 소상공인 경기동향(BSI) 지수 (소상공인, 부문별 실적 및 전망) #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=s0+s1+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=142&tblId=DT_S0001N_001'
sBsi_Q = mergeTimeSeriesDataFrame(dataRefine(urlRequest(url)))

# '소상공인 부문별 실적 및 전망_경기전반_체감' : 'sSenOverallEcono'
# '소상공인 부문별 실적 및 전망_매출_체감'	 : 'sSenRevenue'
# '소상공인 부문별 실적 및 전망_영업이익_체감' : 'sSenOpProfit'	
# '소상공인 부문별 실적 및 전망_자금사정_체감' : 'sSenMoney'
# '소상공인 부문별 실적 및 전망_원재료조달사정_체감' : 'sSenProcure'	
# '소상공인 부문별 실적 및 전망_재고_체감' : 'sSenStock'
# '소상공인 부문별 실적 및 전망_고용_체감' : 'sSenEmployee'
# '소상공인 부문별 실적 및 전망_비용상황_체감' : 'sSenCost'
# '소상공인 부문별 실적 및 전망_구매고객수_체감' : 'sSenCustomer'
# '소상공인 부문별 실적 및 전망_경기전반_전망' : 'sPredOverallEcono'
# '소상공인 부문별 실적 및 전망_매출_전망'	 : 'sPredRevenue'
# '소상공인 부문별 실적 및 전망_영업이익_전망' : 'sPredOpProfit'	
# '소상공인 부문별 실적 및 전망_자금사정_전망' : 'sPredMoney'
# '소상공인 부문별 실적 및 전망_원재료조달사정_전망' : 'sPredProcure'	
# '소상공인 부문별 실적 및 전망_재고_전망' : 'sPredStock'
# '소상공인 부문별 실적 및 전망_고용_전망' : 'sPredEmployee'
# '소상공인 부문별 실적 및 전망_비용상황_전망' : 'sPredCost'
# '소상공인 부문별 실적 및 전망_구매고객수_전망' : 'sPredCustomer'
sBsi_Q.columns = [
    'PRD_DE', 
    'sSenOverallEcono', 'sSenRevenue', 'sSenOpProfit', 'sSenMoney', 'sSenProcure', 'sSenStock', 'sSenEmployee', 'sSenCost', 'sSenCustomer',
    'sPredOverallEcono', 'sPredRevenue', 'sPredOpProfit', 'sPredMoney', 'sPredProcure', 'sPredStock', 'sPredEmployee', 'sPredCost', 'sPredCustomer'
]

# 선행지수 만들기 : 3/6/9개월 선행지수를 만들고, (_3M, _6M, _9M)가 붙은 컬럼을 만듦 
columns = list(sBsi_Q.columns)
# lag 생성
for col in sBsi_Q.columns[1:]:
    # 현재 컬럼의 위치 index
    idx = sBsi_Q.columns.get_loc(col) + 1  
    for lag in range(1, 5):  # lag_1 ~ lag_4
        lag_col_name = f"{col}_{lag*3}M"
        sBsi_Q.insert(loc=idx, column=lag_col_name, value=sBsi_Q[col].shift(lag))
        idx += 1  # 다음 lag 컬럼은 그 다음에 삽입
display(sBsi_Q)

# TIME컬럼 중 'Q4'로 끝나는 row만 남기고, TIME컬럼의 'Q4'를 모두 지워서 'YYYY'형태의 데이터만 남김
# 'Q4'로 끝나는 row만 필터링
df_q4 = sBsi_Q[sBsi_Q['PRD_DE'].str.endswith('Q4')].copy()
df_q4['PRD_DE'] = df_q4['PRD_DE'].str.replace('Q4', '', regex=False)
sBsi = df_q4.copy()

display(sBsi)
save_excel_and_csv(sBsi, 'kosis소상공인BSI')
#########################################################################################################

,PRD_DE,sSenOverallEcono,sSenOverallEcono_3M,sSenOverallEcono_6M,sSenOverallEcono_9M,sSenOverallEcono_12M,sSenRevenue,sSenRevenue_3M,sSenRevenue_6M,sSenRevenue_9M,...,sPredCost,sPredCost_3M,sPredCost_6M,sPredCost_9M,sPredCost_12M,sPredCustomer,sPredCustomer_3M,sPredCustomer_6M,sPredCustomer_9M,sPredCustomer_12M
0,2018Q1,79.7,None,None,None,None,80.1,None,None,None,...,NaN,None,None,None,None,NaN,None,None,None,None
1,2018Q2,62.2,79.7,None,None,None,61.7,80.1,None,None,...,NaN,NaN,None,None,None,NaN,NaN,None,None,None
2,2018Q3,67.9,62.2,79.7,None,None,68.2,61.7,80.1,None,...,NaN,NaN,NaN,None,None,NaN,NaN,NaN,None,None
3,2018Q4,62.1,67.9,62.2,79.7,None,61.9,68.2,61.7,80.1,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None
4,2019Q1,73.3,62.1,67.9,62.2,79.7,73.3,61.9,68.2,61.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2019Q2,65.6,73.3,62.1,67.9,62.2,65.8,73.3,61.9,68.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2019Q3,76.4,65.6,73.3,62.1,67.9,76.5,65.8,73.3,61.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2019Q4,66.4,76.4,65.6,73.3,62.1,64.8,76.5,65.8,73.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020Q1,29.7,66.4,76.4,65.6,73.3,29.3,64.8,76.5,65.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020Q2,82.6,29.7,66.4,76.4,65.6,82.4,29.3,64.8,76.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,PRD_DE,sSenOverallEcono,sSenOverallEcono_3M,sSenOverallEcono_6M,sSenOverallEcono_9M,sSenOverallEcono_12M,sSenRevenue,sSenRevenue_3M,sSenRevenue_6M,sSenRevenue_9M,...,sPredCost,sPredCost_3M,sPredCost_6M,sPredCost_9M,sPredCost_12M,sPredCustomer,sPredCustomer_3M,sPredCustomer_6M,sPredCustomer_9M,sPredCustomer_12M
3,2018,62.1,67.9,62.2,79.7,None,61.9,68.2,61.7,80.1,...,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None
7,2019,66.4,76.4,65.6,73.3,62.1,64.8,76.5,65.8,73.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,2020,51.6,54.9,82.6,29.7,66.4,51.3,54.2,82.4,29.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,2021,39.3,57.6,53.6,59.2,51.6,41.6,58.8,51.5,56.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2022,56.5,71.6,65.3,54.4,39.3,56.8,72.7,66.6,52,...,106.6,108.5,106.4,98.5,NaN,82.9,88.6,85.6,79.4,NaN


In [197]:
# 소상공인 경기동향(BSI) 지수 (전통시장, 부문별 실적 및 전망)

# dataRefine 함수
def dataRefine(refDf):
    tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # API Data를 DataFrame형태로 저장할 딕셔너리
    split_dfs = {}
    # Row로 구분되어 있는 조건들(예, '체감', '전망')을 개개의 컬럼으로 만들어주기 위한 준비
    filter_1 = tempDf['ITM_NM'].drop_duplicates().values.tolist()
    filter_2 = tempDf['C1_NM'].drop_duplicates().values.tolist()
    # 조건 조합 반복
    for s in filter_1:
        for p in filter_2:
            # 'DT'컬럼의 이름을 대체함
            newColName = f"{tempDf['TBL_NM'].unique()[0][:]}_{p}_{s}"  
            filtered = tempDf[(tempDf['ITM_NM'] == s) & (tempDf['C1_NM'] == p)]
            split_dfs[newColName] = filtered.copy()
            split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
            split_dfs[newColName]['PRD_DE'] = split_dfs[newColName]['PRD_DE'].apply(month_to_quarter)
            # Unknown이 들어가있는 행을 삭제하여 Quater 데이터만 남김
            split_dfs[newColName] = split_dfs[newColName][~split_dfs[newColName]['PRD_DE'].str.contains('Unknown')]
    return split_dfs
 
def mergeTimeSeriesDataFrame(split_dfs):
    df = initQuaterDf()
    df_merged = df.copy()
    for name, sub in split_dfs.items():
        sub_unique = sub.drop_duplicates(subset=['PRD_DE'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 4]], on='PRD_DE', how='left', suffixes=('', '_dup'))  # 필요에 따라 'outer' 등으로 변경
    return df_merged
    

##################### 소상공인 경기동향(BSI) 지수 (전통시장, 부문별 실적 및 전망) #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=s0+s1+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=142&tblId=DT_S0001N_003'
mBsi_Q = mergeTimeSeriesDataFrame(dataRefine(urlRequest(url)))

# '전통시장 부문별 실적 및 전망_경기전반_체감' : 'mSenOverallEcono'
# '전통시장 부문별 실적 및 전망_매출_체감'	 : 'mSenRevenue'
# '전통시장 부문별 실적 및 전망_영업이익_체감' : 'mSenOpProfit'	
# '전통시장 부문별 실적 및 전망_자금사정_체감' : 'mSenMoney'
# '전통시장 부문별 실적 및 전망_원재료조달사정_체감' : 'mSenProcure'	
# '전통시장 부문별 실적 및 전망_재고_체감' : 'mSenStock'
# '전통시장 부문별 실적 및 전망_고용_체감' : 'mSenEmployee'
# '전통시장 부문별 실적 및 전망_비용상황_체감' : 'mSenCost'
# '전통시장 부문별 실적 및 전망_구매고객수_체감' : 'mSenCustomer'
# '전통시장 부문별 실적 및 전망_경기전반_전망' : 'mPredOverallEcono'
# '전통시장 부문별 실적 및 전망_매출_전망'	 : 'mPredRevenue'
# '전통시장 부문별 실적 및 전망_영업이익_전망' : 'mPredOpProfit'	
# '전통시장 부문별 실적 및 전망_자금사정_전망' : 'mPredMoney'
# '전통시장 부문별 실적 및 전망_원재료조달사정_전망' : 'mPredProcure'	
# '전통시장 부문별 실적 및 전망_재고_전망' : 'mPredStock'
# '전통시장 부문별 실적 및 전망_고용_전망' : 'mPredEmployee'
# '전통시장 부문별 실적 및 전망_비용상황_전망' : 'mPredCost'
# '전통시장 부문별 실적 및 전망_구매고객수_전망' : 'mPredCustomer'
mBsi_Q.columns = [
    'PRD_DE', 
    'mSenOverallEcono', 'mSenRevenue', 'mSenOpProfit', 'mSenMoney', 'mSenProcure', 'mSenStock', 'mSenEmployee', 'mSenCost', 'mSenCustomer',
    'mPredOverallEcono', 'mPredRevenue', 'mPredOpProfit', 'mPredMoney', 'mPredProcure', 'mPredStock', 'mPredEmployee', 'mPredCost', 'mPredCustomer'
]

# 선행지수 만들기 : 3/6/9개월 선행지수를 만들고, (_3M, _6M, _9M)가 붙은 컬럼을 만듦 
columns = list(mBsi_Q.columns)
# lag 생성
for col in mBsi_Q.columns[1:]:
    # 현재 컬럼의 위치 index
    idx = mBsi_Q.columns.get_loc(col) + 1  
    for lag in range(1, 4):  # lag_1 ~ lag_3
        lag_col_name = f"{col}_{lag*4}M"
        mBsi_Q.insert(loc=idx, column=lag_col_name, value=mBsi_Q[col].shift(lag))
        idx += 1  # 다음 lag 컬럼은 그 다음에 삽입
display(mBsi_Q)

# TIME컬럼 중 'Q4'로 끝나는 row만 남기고, TIME컬럼의 'Q4'를 모두 지워서 'YYYY'형태의 데이터만 남김
# 'Q4'로 끝나는 row만 필터링
df_q4 = mBsi_Q[mBsi_Q['PRD_DE'].str.endswith('Q4')].copy()
df_q4['PRD_DE'] = df_q4['PRD_DE'].str.replace('Q4', '', regex=False)
mBsi = df_q4.copy()

display(mBsi)
save_excel_and_csv(mBsi, 'kosis전통시장BSI')
####################################################################################################

,PRD_DE,mSenOverallEcono,mSenOverallEcono_4M,mSenOverallEcono_8M,mSenOverallEcono_12M,mSenRevenue,mSenRevenue_4M,mSenRevenue_8M,mSenRevenue_12M,mSenOpProfit,...,mPredEmployee_8M,mPredEmployee_12M,mPredCost,mPredCost_4M,mPredCost_8M,mPredCost_12M,mPredCustomer,mPredCustomer_4M,mPredCustomer_8M,mPredCustomer_12M
0,2018Q1,67,None,None,None,65.8,None,None,None,65.7,...,None,None,NaN,None,None,None,NaN,None,None,None
1,2018Q2,54.1,67,None,None,53.2,65.8,None,None,53.1,...,None,None,NaN,NaN,None,None,NaN,NaN,None,None
2,2018Q3,73.2,54.1,67,None,73.2,53.2,65.8,None,73,...,NaN,None,NaN,NaN,NaN,None,NaN,NaN,NaN,None
3,2018Q4,52.7,73.2,54.1,67,52.7,73.2,53.2,65.8,52.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019Q1,62.4,52.7,73.2,54.1,62.1,52.7,73.2,53.2,62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2019Q2,57,62.4,52.7,73.2,56.9,62.1,52.7,73.2,56.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2019Q3,84.6,57,62.4,52.7,84.2,56.9,62.1,52.7,84.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2019Q4,65.4,84.6,57,62.4,65.6,84.2,56.9,62.1,65.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020Q1,28.4,65.4,84.6,57,28.3,65.6,84.2,56.9,NaN,...,NaN,NaN,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020Q2,79.2,28.4,65.4,84.6,78.2,28.3,65.6,84.2,NaN,...,NaN,NaN,100,98,NaN,NaN,NaN,NaN,NaN,NaN


,PRD_DE,mSenOverallEcono,mSenOverallEcono_4M,mSenOverallEcono_8M,mSenOverallEcono_12M,mSenRevenue,mSenRevenue_4M,mSenRevenue_8M,mSenRevenue_12M,mSenOpProfit,...,mPredEmployee_8M,mPredEmployee_12M,mPredCost,mPredCost_4M,mPredCost_8M,mPredCost_12M,mPredCustomer,mPredCustomer_4M,mPredCustomer_8M,mPredCustomer_12M
3,2018,52.7,73.2,54.1,67,52.7,73.2,53.2,65.8,52.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2019,65.4,84.6,57,62.4,65.6,84.2,56.9,62.1,65.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,2020,44.8,65.1,79.2,28.4,44.7,64.8,78.2,28.3,NaN,...,100,100.7,100,99.2,100,98,NaN,NaN,NaN,NaN
15,2021,41.2,77.4,49.2,44.5,42.8,82.4,47.9,42.8,NaN,...,108.9,106.2,100.2,98.3,100.1,99.2,NaN,NaN,NaN,NaN
19,2022,54,79,62.6,40.3,53.1,79.5,60.7,37,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,104,110.9,106.4,98.3


In [198]:
# 소상공인 경기동향(BSI) 지수 (소상공인, 업종별 실적 및 전망)
# 소상공인 경기동향(BSI) 지수 (소상공인, 지역별 실적 및 전망)

# dataRefine 함수
def dataRefine(refDf):
    temp_df = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # API Data를 DataFrame형태로 저장할 딕셔너리
    split_dfs = {}
    # Row로 구분되어 있는 조건들(예, '체감', '전망')을 개개의 컬럼으로 만들어주기 위한 준비
    filter_1 = temp_df['ITM_NM'].drop_duplicates().values.tolist()
    # 조건 조합 반복
    for s in filter_1:
        # 'DT'컬럼의 이름을 대체함
        newColName = f"{temp_df['TBL_NM'].unique()[0][:]}_{s}"        
        filtered = temp_df[temp_df['ITM_NM'] == s]
        split_dfs[newColName] = filtered.copy()
        split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
        split_dfs[newColName]['PRD_DE'] = split_dfs[newColName]['PRD_DE'].apply(month_to_quarter)
        # Unknown이 들어가있는 행을 삭제하여 Quater 데이터만 남김
        split_dfs[newColName] = split_dfs[newColName][~split_dfs[newColName]['PRD_DE'].str.contains('Unknown')]
    return split_dfs

def mergeDataFrame(refDf):
    first_df = list(refDf.values())[0]
    df_merged = first_df[['PRD_DE', 'C1_NM']].copy()
    for name, sub in refDf.items():
        sub_unique = sub.drop_duplicates(subset=['PRD_DE', 'C1_NM'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 3, 4]], on=['PRD_DE', 'C1_NM'], how='left')  
    return df_merged
    
def mergeTimeSeriesDataFrame(refDf):
    df = initQuaterDf()
    return df.merge(refDf, on='PRD_DE', how='left')  


#################### 소상공인 경기동향(BSI) 지수 (소상공인, 업종별 실적 및 전망) ####################
# 업종 매핑 불가능
# url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=s0+s1+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=142&tblId=DT_S0001N_002'
# msInduBsiDf = mergeTimeSeriesDataFrame(mergeDataFrame(dataRefine(urlRequest(url))))
# display(msInduBsiDf)
# save_excel_and_csv(msInduBsiDf, 'msInduBsiDf')
####################################################################################################


##################### 소상공인 경기동향(BSI) 지수 (소상공인, 지역별 실적 및 전망) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=s0+s1+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=142&tblId=DT_S0001N_005'
sLocalBsi_Q = mergeTimeSeriesDataFrame(mergeDataFrame(dataRefine(urlRequest(url))))

# 소상공인 지역별 실적 및 전망_체감 : sRegionSenBSI
# 소상공인 지역별 실적 및 전망_전망 : sRegionPredBSI
sLocalBsi_Q.columns = [
    'PRD_DE', 'REGION', 
    'sRegionSenBSI', 'sRegionPredBSI'
]

display(sLocalBsi_Q)

# 선행지수 만들기 : 3/6/9개월 선행지수를 만들고, (_3M, _6M, _9M)가 붙은 컬럼을 만듦 
# lag 생성할 대상 컬럼
target_cols = ['sRegionSenBSI', 'sRegionPredBSI']
lag_n = 4  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = sLocalBsi_Q.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{3*lag}M'
        # REGION 기준으로 그룹별 shift
        lag_series = sLocalBsi_Q.groupby('REGION')[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        sLocalBsi_Q.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

# 'Q4'로 끝나는 row만 필터링
df_q4 = sLocalBsi_Q[sLocalBsi_Q['PRD_DE'].str.endswith('Q4')].copy()
df_q4['PRD_DE'] = df_q4['PRD_DE'].str.replace('Q4', '', regex=False)
sLocalBsi = df_q4.copy()

display(sLocalBsi)
save_excel_and_csv(sLocalBsi, 'kosis소상공인BSI_지역')
####################################################################################################

,PRD_DE,REGION,sRegionSenBSI,sRegionPredBSI
0,2018Q1,서울,77.8,106
1,2018Q1,부산,78.9,98.8
2,2018Q1,대구,70.2,97.7
3,2018Q1,인천,84.3,102.1
4,2018Q1,광주,85.8,106.2
...,...,...,...,...
335,2022Q4,전북,48.4,86.1
336,2022Q4,전남,50.8,82.2
337,2022Q4,경북,58.8,86.3
338,2022Q4,경남,59.9,80.8


,PRD_DE,REGION,sRegionSenBSI,sRegionSenBSI_3M,sRegionSenBSI_6M,sRegionSenBSI_9M,sRegionSenBSI_12M,sRegionPredBSI,sRegionPredBSI_3M,sRegionPredBSI_6M,sRegionPredBSI_9M,sRegionPredBSI_12M
51,2018,서울,57.5,67.2,62.6,77.8,NaN,86.6,98.5,82.8,106,NaN
52,2018,부산,59.6,64.3,55.8,78.9,NaN,82.7,93.3,82.7,98.8,NaN
53,2018,대구,59.3,58.6,54.6,70.2,NaN,86.1,90.4,86.1,97.7,NaN
54,2018,인천,58.7,66.4,62.6,84.3,NaN,88.8,93,80.4,102.1,NaN
55,2018,광주,71.2,75.7,69,85.8,NaN,87.6,108,80.1,106.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
335,2022,전북,48.4,77,63.9,52,37.5,86.1,86.5,85.7,87.3,87.5
336,2022,전남,50.8,71.1,62.8,46.7,42.7,82.2,93.4,78.9,77.3,89.4
337,2022,경북,58.8,72.2,69.6,52.3,36.5,86.3,91.5,92.2,80.1,85.2
338,2022,경남,59.9,73,68.6,54.4,36.5,80.8,85.8,81.4,83.1,85.7


In [199]:
# 소상공인 경기동향(BSI) 지수 (전통시장, 업종별 실적 및 전망)
# 소상공인 경기동향(BSI) 지수 (전통시장, 지역별 실적 및 전망)

# dataRefine 함수
def dataRefine(refDf):
    temp_df = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # API Data를 DataFrame형태로 저장할 딕셔너리
    split_dfs = {}
    # Row로 구분되어 있는 조건들(예, '체감', '전망')을 개개의 컬럼으로 만들어주기 위한 준비
    filter_1 = temp_df['ITM_NM'].drop_duplicates().values.tolist()
    # 조건 조합 반복
    for s in filter_1:
        # 'DT'컬럼의 이름을 대체함
        newColName = f"{temp_df['TBL_NM'].unique()[0][:]}_{s}"        
        filtered = temp_df[temp_df['ITM_NM'] == s]
        split_dfs[newColName] = filtered.copy()
        split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
        split_dfs[newColName]['PRD_DE'] = split_dfs[newColName]['PRD_DE'].apply(month_to_quarter)
        # Unknown이 들어가있는 행을 삭제하여 Quater 데이터만 남김
        split_dfs[newColName] = split_dfs[newColName][~split_dfs[newColName]['PRD_DE'].str.contains('Unknown')]
    return split_dfs

def mergeDataFrame(refDf):
    first_df = list(refDf.values())[0]
    df_merged = first_df[['PRD_DE', 'C1_NM']].copy()
    for name, sub in refDf.items():
        sub_unique = sub.drop_duplicates(subset=['PRD_DE', 'C1_NM'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 3, 4]], on=['PRD_DE', 'C1_NM'], how='left')  
    return df_merged
    
def mergeTimeSeriesDataFrame(refDf):
    df = initQuaterDf()
    return df.merge(refDf, on='PRD_DE', how='left')  


##################### 소상공인 경기동향(BSI) 지수 (전통시장, 업종별 실적 및 전망) #####################
# 업종 매핑 불가능
# url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=s0+s1+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=142&tblId=DT_S0001N_004'
# traditionalMarketInduBsiDf = mergeTimeSeriesDataFrame(mergeDataFrame(dataRefine(urlRequest(url))))
# display(traditionalMarketInduBsiDf)
# save_excel_and_csv(traditionalMarketInduBsiDf, 'traditionalMarketInduBsiDf')
#########################################################################################################


###################### 소상공인 경기동향(BSI) 지수 (전통시장, 지역별 실적 및 전망) #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=s0+s1+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=142&tblId=DT_S0001N_006'
mLocalBsi_Q = mergeTimeSeriesDataFrame(mergeDataFrame(dataRefine(urlRequest(url))))

# 전통시장 지역별 실적 및 전망_체감 : mRegionSenBSI
# 전통시장 지역별 실적 및 전망_전망 : mRegionPredBSI
mLocalBsi_Q.columns = [
    'PRD_DE', 'REGION', 
    'mRegionSenBSI', 'mRegionPredBSI'
]
display(mLocalBsi_Q)

# lag 생성할 대상 컬럼
target_cols = ['mRegionSenBSI', 'mRegionPredBSI']
lag_n = 4  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = mLocalBsi_Q.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{3*lag}M'
        # REGION 기준으로 그룹별 shift
        lag_series = mLocalBsi_Q.groupby('REGION')[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        mLocalBsi_Q.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

# 'Q4'로 끝나는 row만 필터링
df_q4 = mLocalBsi_Q[mLocalBsi_Q['PRD_DE'].str.endswith('Q4')].copy()
df_q4['PRD_DE'] = df_q4['PRD_DE'].str.replace('Q4', '', regex=False)
mLocalBsi = df_q4.copy()

display(mLocalBsi)
save_excel_and_csv(mLocalBsi, 'kosis전통시장BSI_지역')
#########################################################################################################

,PRD_DE,REGION,mRegionSenBSI,mRegionPredBSI
0,2018Q1,서울,71.8,94
1,2018Q1,부산,54.2,94.9
2,2018Q1,대구,62.2,84.6
3,2018Q1,인천,70.3,94.2
4,2018Q1,광주,61.2,82.7
...,...,...,...,...
335,2022Q4,전북,52.3,79.7
336,2022Q4,전남,51.2,87.6
337,2022Q4,경북,59.6,76
338,2022Q4,경남,65.7,84.8


,PRD_DE,REGION,mRegionSenBSI,mRegionSenBSI_3M,mRegionSenBSI_6M,mRegionSenBSI_9M,mRegionSenBSI_12M,mRegionPredBSI,mRegionPredBSI_3M,mRegionPredBSI_6M,mRegionPredBSI_9M,mRegionPredBSI_12M
51,2018,서울,42.3,75.7,55.3,71.8,NaN,74.6,116.2,83.1,94,NaN
52,2018,부산,57,70.1,56.1,54.2,NaN,87.4,103.3,72.9,94.9,NaN
53,2018,대구,50.6,70.5,47.4,62.2,NaN,78.2,114.7,74.4,84.6,NaN
54,2018,인천,59.4,80.4,63,70.3,NaN,80.4,108.7,87,94.2,NaN
55,2018,광주,66.3,72.4,56.1,61.2,NaN,91.8,103.1,74.5,82.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
335,2022,전북,52.3,87.5,49.2,49.2,37.9,79.7,108.6,83.6,81.3,86.3
336,2022,전남,51.2,78.8,67.6,39.4,47.8,87.6,99.4,85.9,72.9,85.7
337,2022,경북,59.6,85.1,66.3,36.1,45.4,76,102.4,81.3,75.5,85.2
338,2022,경남,65.7,82.9,59.5,33.8,31.8,84.8,99,83.3,78.1,83.3


In [200]:
# 총인구수(전국/시도/시/군/구)(단위: 명)
# 총가구수(전국/시도/시/군/구)(단위: 가구)
# 총주택수(전국/시도/시/군/구)

# dataRefine 함수
def dataRefine(refDf):
    tempDf = refDf[['ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy() 
    newColName = tempDf['ITM_NM'].unique()[0]          # 'DT' 컬럼 네임 교체
    tempDf.rename(columns={'DT': newColName}, inplace=True)
    return tempDf[['PRD_DE', 'C1_NM', newColName]]

#################### 총인구수(전국/시도/시/군/구)(단위: 명) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T00+&objL1=ALL&objL2=ALL&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=INH_1IN1503_01'
populationDf = dataRefine(urlRequest(url))
################################################################################

##################### 총가구수(전국/시도/시/군/구)(단위: 가구) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T1100+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=INH_1JC1501'
houseHoldDf = dataRefine(urlRequest(url))
##################################################################################

##################### 총주택수(전국/시도/시/군/구) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T10+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=INH_1JU1501'
houseDf = dataRefine(urlRequest(url))
#######################################################################

dfs = [populationDf, houseHoldDf, houseDf]
# 모든 df를 PRD_DE, C1_NM 기준으로 내부 조인
popStat_Y = reduce(lambda left, right: pd.merge(left, right, on=['PRD_DE', 'C1_NM']), dfs)

# 총인구(명) : population
# 총가구_가구 : household
# 주택 : house
popStat_Y.columns = [
    'PRD_DE', 'REGION',
    'population', 'household', 'house'
]
display(popStat_Y)

# 선행지수 만들기 : 1년 선행지수를 만들고, (_1Y)가 붙은 컬럼을 만듦 
# lag 생성할 대상 컬럼
target_cols = ['population', 'household', 'house']
lag_n = 1  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = popStat_Y.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{lag}Y'
        # REGION 기준으로 그룹별 shift
        lag_series = popStat_Y.groupby('REGION')[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        popStat_Y.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

popStat = popStat_Y.copy()
popStat = popStat[popStat['PRD_DE'] > startYYYY]
display(popStat)
save_excel_and_csv(popStat, 'kosis인구가구주택_지역')

,PRD_DE,REGION,population,household,house
0,2017,전국,51422507,20167922,17122573
1,2018,전국,51629512,20499543,17633327
2,2019,전국,51779203,20891348,18126954
3,2020,전국,51829136,21484785,18525844
4,2021,전국,51738071,22022753,18811627
...,...,...,...,...,...
6511,2018,서귀포시,175350,70383,67848
6512,2019,서귀포시,177360,71461,71357
6513,2020,서귀포시,178552,73627,72684
6514,2021,서귀포시,179238,76208,74077


,PRD_DE,REGION,population,population_1Y,household,household_1Y,house,house_1Y
1,2018,전국,51629512,51422507,20499543,20167922,17633327,17122573
2,2019,전국,51779203,51629512,20891348,20499543,18126954,17633327
3,2020,전국,51829136,51779203,21484785,20891348,18525844,18126954
4,2021,전국,51738071,51829136,22022753,21484785,18811627,18525844
5,2022,전국,51692272,51738071,22383187,22022753,19155585,18811627
...,...,...,...,...,...,...,...,...
6511,2018,서귀포시,175350,169358,70383,66746,67848,64421
6512,2019,서귀포시,177360,175350,71461,70383,71357,67848
6513,2020,서귀포시,178552,177360,73627,71461,72684,71357
6514,2021,서귀포시,179238,178552,76208,73627,74077,72684


In [201]:
# 중소기업 경기동행종합지수

# dataRefine 함수
def dataRefine(refDf):
    temp_df = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # API Data를 DataFrame형태로 저장할 딕셔너리
    split_dfs = {}
    # Row로 구분되어 있는 조건들(예, '체감', '전망')을 개개의 컬럼으로 만들어주기 위한 준비
    filter_1 = temp_df['C1_NM'].drop_duplicates().values.tolist()
    # 조건 조합 반복
    for s in filter_1:
        # 'DT'컬럼의 이름을 대체함
        newColName = f"{temp_df['TBL_NM'].unique()[0][:]}_{s}"        
        filtered = temp_df[temp_df['C1_NM'] == s]
        split_dfs[newColName] = filtered.copy()
        split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
        split_dfs[newColName]['PRD_DE'] = split_dfs[newColName]['PRD_DE'].apply(month_to_quarter)
        # Unknown이 들어가있는 행을 삭제하여 Quater 데이터만 남김
        split_dfs[newColName] = split_dfs[newColName][~split_dfs[newColName]['PRD_DE'].str.contains('Unknown')]
    return split_dfs

def mergeDataFrame(refDf):
    first_df = list(refDf.values())[0]
    df_merged = first_df.iloc[:, [2, 4]].copy()
    for name, sub in list(refDf.items())[1:]:
        sub_unique = sub.drop_duplicates(subset=['PRD_DE'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 4]], on=['PRD_DE'], how='left')  
    return df_merged
    
def mergeTimeSeriesDataFrame(refDf):
    df = initQuaterDf()
    return df.merge(refDf, on='PRD_DE', how='left')  


##################### 중소기업 경기동행종합지수 - Coincident Economic Index (CEI) ##################### 
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=303&tblId=DT_303005_CI001'
cei_Q = mergeTimeSeriesDataFrame(mergeDataFrame(dataRefine(urlRequest(url))))

# 중소기업 경기동행종합지수_동행종합지수 : coinEconoIdx
# 중소기업 경기동행종합지수_동행지수 순환변동치 : ceiCycle
# 중소기업 경기동행종합지수_동행지수 전월비 : coinEconoIdx_1M
# 중소기업 경기동행종합지수_동행지수 전월차 : coinEconoIdxDiff
# 중소기업 경기동행종합지수_동행지수 전년동기대비 : coinEconoIdx_12M
# 중소기업 경기동행종합지수_순환변동치 전월비 : ceiCycle_1M
# 중소기업 경기동행종합지수_순환변동치 전월차 : ceiCycleDiff
# 중소기업 경기동행종합지수_순환변동치 전년동기대비 : ceiCycle_12M
cei_Q.columns = [
    'PRD_DE', 
    'coinEconoIdx', 'ceiCycle', 
    'coinEconoIdx_1M', 'coinEconoIdxDiff', 'coinEconoIdx_12M', 
    'ceiCycle_1M', 'ceiCycleDiff', 'ceiCycle_12M'
]
display(cei_Q)

# 'Q4'로 끝나는 row만 필터링
df_q4 = cei_Q[cei_Q['PRD_DE'].str.endswith('Q4')].copy()
df_q4['PRD_DE'] = df_q4['PRD_DE'].str.replace('Q4', '', regex=False)
cei = df_q4.copy()

display(cei)
save_excel_and_csv(cei, 'kosis경기동행지수CEI')
################################################################# 

,PRD_DE,coinEconoIdx,ceiCycle,coinEconoIdx_1M,coinEconoIdxDiff,coinEconoIdx_12M,ceiCycle_1M,ceiCycleDiff,ceiCycle_12M
0,2018Q1,102.43,100.78,0.13,0.13,0.14,0.11,0.11,-0.14
1,2018Q2,102.39,100.69,0.01,0.01,0.06,-0.01,-0.01,-0.18
2,2018Q3,102.13,100.4,-0.07,-0.07,-0.5,-0.08,-0.08,-0.69
3,2018Q4,102.18,100.42,-0.07,-0.07,0.2,-0.07,-0.07,0.03
4,2019Q1,102.39,100.6,0.22,0.22,-0.04,0.22,0.22,-0.18
5,2019Q2,102.47,100.65,-0.12,-0.12,0.08,-0.13,-0.13,-0.04
6,2019Q3,102.26,100.42,-0.17,-0.17,0.13,-0.17,-0.17,0.02
7,2019Q4,102.48,100.6,0.36,0.37,0.29,0.36,0.36,0.18
8,2020Q1,100.47,98.58,-1.27,-1.29,-1.88,-1.28,-1.28,-2.01
9,2020Q2,97.56,95.68,-0.24,-0.23,-4.79,-0.24,-0.23,-4.94


,PRD_DE,coinEconoIdx,ceiCycle,coinEconoIdx_1M,coinEconoIdxDiff,coinEconoIdx_12M,ceiCycle_1M,ceiCycleDiff,ceiCycle_12M
3,2018,102.18,100.42,-0.07,-0.07,0.2,-0.07,-0.07,0.03
7,2019,102.48,100.6,0.36,0.37,0.29,0.36,0.36,0.18
11,2020,100.41,98.31,-0.13,-0.13,-2.02,-0.16,-0.16,-2.28
15,2021,103.1,100.51,0.29,0.3,2.68,0.26,0.26,2.24
19,2022,103.76,100.67,-0.22,-0.23,0.64,-0.26,-0.26,0.16


In [202]:
# 종업원 규모별 설비투자(서비스업, 300명 미만 사업장)(단위: 억원)
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T1+&objL1=001001001+001001002+001001003+001001004+001001005+001001006+001001007+001001008+001001009+001001010+001001011+001001011001+001001011002+001001011003+001001011004+001001012+001001013+001001013001+001001013002+001001014+001001014001+001001014002+001001014003+001001015+001001015001+001001015002+001001015003+001001016+001001017+001001017001+001001017002+001001017003+001001017004+001001017005+001001018+001001019+001001019001+001001019002+001001019003+001001019004+001001020+001001020001+001001020002+001001021+001001021001+001001021002+001001022+001001022001+001001022002+001001023+001001024+001002+001002001+001002002+001002003+001002004+001002005+001002005001+001002005002+001002005003+001002006+001002006001+001002006002+001002006003+001002006004+001002007+001002007001+001002007002+001002008+001002008001+001002008002+001002008003+001002008004+001002008005+001002008006+001002009+001002010+001002011+&objL2=000+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=406&tblId=DT_406N_302001_A006'
df = urlRequest(url)
tempDf = df[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()

# 'DT' 컬럼 네임을 '종업원 규모별 설비투자'으로 교체
newColName = tempDf['ITM_NM'].unique()[0]
tempDf.rename(columns={'DT': newColName}, inplace=True)
capitalSpend_Y = tempDf[['PRD_DE', 'C1_NM', newColName]]
capitalSpend_Y = capitalSpend_Y[capitalSpend_Y['C1_NM'].isin(['도매 및 소매업', '운수 및 창고업', '숙박 및 음식점업', '부동산업', '보건업 및 사회복지 서비스업'])]

# 총인구(명) : population
# 총가구_가구 : household
# 주택 : house
capitalSpend_Y.columns = [
    'PRD_DE', 'KSIC', 'capSpend'
]
display(capitalSpend_Y)

# 선행지수 만들기 : 1년 선행지수를 만들고, (_1Y)가 붙은 컬럼을 만듦 
# lag 생성할 대상 컬럼
target_cols = ['capSpend']
lag_n = 1  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = capitalSpend_Y.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{lag}Y'
        # REGION 기준으로 그룹별 shift
        lag_series = capitalSpend_Y.groupby('KSIC')[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        capitalSpend_Y.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

replace_dict = {
    '도매 및 소매업': 'G', 
    '운수 및 창고업': 'H', 
    '숙박 및 음식점업': 'I', 
    '부동산업': 'L', 
    '보건업 및 사회복지 서비스업': 'Q'
}
capitalSpend_Y['KSIC'] = capitalSpend_Y['KSIC'].replace(replace_dict)
capitalSpend_Y[capitalSpend_Y['KSIC'].isin(list(replace_dict.values()))]
capitalSpend = capitalSpend_Y.copy()
capitalSpend = capitalSpend[capitalSpend['PRD_DE'] > startYYYY]
display(capitalSpend)
save_excel_and_csv(capitalSpend, 'kosis설비투자_업종')

,PRD_DE,KSIC,capSpend
112,2021,도매 및 소매업,2984.38978629
113,2022,도매 및 소매업,13970.353258
120,2021,운수 및 창고업,20737.87817393
121,2022,운수 및 창고업,15476.427168
130,2021,숙박 및 음식점업,4316.10413337
131,2022,숙박 및 음식점업,1605.212312
150,2021,부동산업,16672.67941016
151,2022,부동산업,770.419643
154,2021,보건업 및 사회복지 서비스업,1100.82727273
155,2022,보건업 및 사회복지 서비스업,16.766667


,PRD_DE,KSIC,capSpend,capSpend_1Y
112,2021,G,2984.38978629,NaN
113,2022,G,13970.353258,2984.38978629
120,2021,H,20737.87817393,NaN
121,2022,H,15476.427168,20737.87817393
130,2021,I,4316.10413337,NaN
131,2022,I,1605.212312,4316.10413337
150,2021,L,16672.67941016,NaN
151,2022,L,770.419643,16672.67941016
154,2021,Q,1100.82727273,NaN
155,2022,Q,16.766667,1100.82727273


In [203]:
# 경기전반 실적 SBHI
# 경기전반 전망 SBHI
# 내수판매 실적 SBHI
# 수출실적 SBHI
# 수출전망 SBHI
# 영업이익 실적 SBHI
# 영업이익 전망 SBHI
# 자금사정 실적 SBHI
# 자금사정 전망 SBHI
# 고용수준 실적 SBHI
# 고용수준 전망 SBHI

# dataRefine 함수
def dataRefine(refDf):
    temp_df = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # 'DT' 컬럼 네임을 'TBL_NM'으로 교체
    newColName = temp_df['TBL_NM'].unique()[0]
    temp_df.rename(columns={'DT': newColName}, inplace=True)
    temp_df[['PRD_DE', 'C1_NM', newColName]]
    # 월 → 분기(Q1~Q4)로 변환 : 3,6,9,12월 자료만 Quater로 변환하고 나머지는 Unknown 처리
    temp_df['PRD_DE'] = temp_df['PRD_DE'].apply(month_to_quarter)
    # Unknown이 들어가있는 행을 삭제하여 Quater 데이터만 남김
    return temp_df[~temp_df['PRD_DE'].str.contains('Unknown')]

def mergeTimeSeriesDataFrame(refDf):
    df = initQuaterDf()
    return  df.merge(refDf, on='PRD_DE', how='left') 

##################### 경기전반 실적 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10100'
refinedDf = dataRefine(urlRequest(url))
econoPerfDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
###############################################################

###################### 경기전반 전망 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10101'
refinedDf = dataRefine(urlRequest(url))
econoPredDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
###############################################################

###################### 내수판매 실적 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10104'
refinedDf = dataRefine(urlRequest(url))
domePerfDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
###############################################################

###################### 내수판매 전망 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10105'
refinedDf = dataRefine(urlRequest(url))
domePredDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
###############################################################

###################### 수출실적 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10106'
refinedDf = dataRefine(urlRequest(url))
expPerfDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
###############################################################

###################### 수출전망 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10107'
refinedDf = dataRefine(urlRequest(url))
expPredDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
###############################################################

###################### 영업이익 실적 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10108'
refinedDf = dataRefine(urlRequest(url))
opPerfDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
###############################################################

###################### 영업이익 전망 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10109'
refinedDf = dataRefine(urlRequest(url))
opPredDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
###############################################################

###################### 자금사정 실적 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10110'
refinedDf = dataRefine(urlRequest(url))
fundPerfDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
###############################################################

###################### 자금사정 전망 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10111'
refinedDf = dataRefine(urlRequest(url))
fundPredDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
###############################################################

###################### 고용수준 실적 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10118'
refinedDf = dataRefine(urlRequest(url))
empPerfDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
###############################################################

###################### 고용수준 전망 SBHI #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=1634013103124559T6+&objL1=15340a.dd+15340a.ddd+15340a.ddd1+15340a.ddd2+15340a.ddd3+15340a.ddd4+15340a.ddd5+15340a.ddd6+15340a.ddd7+15340a.ddd8+15340a.ddd9+15340a.ddd10+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=M&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=340&tblId=DT_B10119'
refinedDf = dataRefine(urlRequest(url))
empPredDf = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
###############################################################

dfs = [econoPerfDf, econoPredDf, domePerfDf, domePredDf, expPerfDf, expPredDf, opPerfDf, opPredDf, fundPerfDf, fundPredDf, empPerfDf, empPredDf]
# 모든 df를 PRD_DE, C1_NM 기준으로 내부 조인
econoSBHI_Q = reduce(lambda left, right: pd.merge(left, right, on=['PRD_DE', 'C1_NM']), dfs)

# 경기전반 실적 SBHI	: econoPerfSBHI
# 경기전반 전망 SBHI	: econoPredSBHI
# 내수판매 실적 SBHI	: domePerfSBHI
# 내수판매 전망 SBHI	: domePredSBHI
# 수출실적 SBHI	: expPerfSBHI
# 수출전망 SBHI	: expPredSBHI
# 영업이익 실적 SBHI	: opPerfSBHI
# 영업이익 전망 SBHI	: opPredSBHI
# 자금사정 실적 SBHI	: fundPerfSBHI
# 자금사정 전망 SBHI	: fundPredSBHI
# 고용수준 실적 SBHI	: empPerfSBHI
# 고용수준 전망 SBHI : empPredSBHI
econoSBHI_Q.columns = [
    'PRD_DE', 'KSIC',
    'econoPerfSBHI', 'econoPredSBHI', 'domePerfSBHI', 'domePredSBHI', 'expPerfSBHI', 'expPredSBHI',
    'opPerfSBHI', 'opPredSBHI', 'fundPerfSBHI', 'fundPredSBHI', 'empPerfSBHI', 'empPredSBHI'
]

# 경기 선행지수 만들기
# lag 생성할 대상 컬럼
target_cols = [
    'econoPerfSBHI', 'econoPredSBHI', 'domePerfSBHI', 'domePredSBHI', 'expPerfSBHI', 'expPredSBHI',
    'opPerfSBHI', 'opPredSBHI', 'fundPerfSBHI', 'fundPredSBHI', 'empPerfSBHI', 'empPredSBHI'
]
lag_n = 4  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = econoSBHI_Q.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{3*lag}M'
        # REGION 기준으로 그룹별 shift
        lag_series = econoSBHI_Q.groupby('KSIC')[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        econoSBHI_Q.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

#print(econoSBHI_Q['KSIC'].unique())
replace_dict = {
    '도매 및 소매업': 'G', 
    '운수업': 'H', 
    '숙박 및 음식점업': 'I', 
    '부동산업 및 임대업': 'L', 
    '사업시설관리 및 사업지원서비스업': 'N',
    '교육서비스업': 'P',
    '보건업 및 사회복지 서비스업': 'Q',
    '예술스포츠 및 여가관련서비스업': 'R',
    '수리 및 기타개인서비스업': 'S'
}
econoSBHI_Q['KSIC'] = econoSBHI_Q['KSIC'].replace(replace_dict)
econoSBHI_Q = econoSBHI_Q[econoSBHI_Q['KSIC'].isin(list(replace_dict.values()))]
display(econoSBHI_Q)

# 'Q4'로 끝나는 row만 필터링
df_q4 = econoSBHI_Q[econoSBHI_Q['PRD_DE'].str.endswith('Q4')].copy()
df_q4['PRD_DE'] = df_q4['PRD_DE'].str.replace('Q4', '', regex=False)
econoSBHI = df_q4.copy()

display(econoSBHI)
save_excel_and_csv(econoSBHI, 'kosis경기선행지수SBHI_업종')

,PRD_DE,KSIC,econoPerfSBHI,econoPerfSBHI_3M,econoPerfSBHI_6M,econoPerfSBHI_9M,econoPerfSBHI_12M,econoPredSBHI,econoPredSBHI_3M,econoPredSBHI_6M,...,empPerfSBHI,empPerfSBHI_3M,empPerfSBHI_6M,empPerfSBHI_9M,empPerfSBHI_12M,empPredSBHI,empPredSBHI_3M,empPredSBHI_6M,empPredSBHI_9M,empPredSBHI_12M
1,2018Q1,G,89.9,NaN,NaN,NaN,NaN,87.8,NaN,NaN,...,92.4,NaN,NaN,NaN,NaN,95.3,NaN,NaN,NaN,NaN
2,2018Q1,S,73.5,NaN,NaN,NaN,NaN,92.6,NaN,NaN,...,93,NaN,NaN,NaN,NaN,90.9,NaN,NaN,NaN,NaN
3,2018Q1,H,83.7,NaN,NaN,NaN,NaN,91.2,NaN,NaN,...,89.3,NaN,NaN,NaN,NaN,89.8,NaN,NaN,NaN,NaN
4,2018Q1,I,73.3,NaN,NaN,NaN,NaN,89.3,NaN,NaN,...,103.9,NaN,NaN,NaN,NaN,101.5,NaN,NaN,NaN,NaN
6,2018Q1,L,84.5,NaN,NaN,NaN,NaN,85.5,NaN,NaN,...,94.9,NaN,NaN,NaN,NaN,95.2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,2022Q4,I,85.3,81.1,94.3,61.2,50,95.9,80.5,94.1,...,91.6,96.5,95.7,93.1,98.1,94.7,93.5,91.1,97.6,91.9
215,2022Q4,L,70.1,68.2,74.4,71.8,68.4,67.4,73.2,83.1,...,100.2,98.5,94,94.7,91.9,94.6,96.7,94,95.8,93
217,2022Q4,N,85.8,85.1,89.5,87,87.3,88.9,88,89.8,...,83.9,83.7,91.3,92.9,89.3,83.8,86.7,90,87.5,90
218,2022Q4,P,76.7,75.8,74.3,75,79.3,87.2,76.5,78.8,...,99.2,103.1,103.1,105.7,100.7,99.2,105.3,103.8,102.2,97.8


,PRD_DE,KSIC,econoPerfSBHI,econoPerfSBHI_3M,econoPerfSBHI_6M,econoPerfSBHI_9M,econoPerfSBHI_12M,econoPredSBHI,econoPredSBHI_3M,econoPredSBHI_6M,...,empPerfSBHI,empPerfSBHI_3M,empPerfSBHI_6M,empPerfSBHI_9M,empPerfSBHI_12M,empPredSBHI,empPredSBHI_3M,empPredSBHI_6M,empPredSBHI_9M,empPredSBHI_12M
34,2018,G,78.7,83.4,86.1,89.9,NaN,87,89.8,89.7,...,94.8,95.5,93.2,92.4,NaN,92.9,94,95.6,95.3,NaN
35,2018,S,74.1,73.7,77.5,73.5,NaN,82.5,85.8,84.4,...,94.7,95.7,92.5,93,NaN,92.9,90.6,92.7,90.9,NaN
36,2018,H,77.4,85.3,81.4,83.7,NaN,81.6,90.3,85.8,...,88.6,87.6,89.4,89.3,NaN,91,90.2,91.1,89.8,NaN
37,2018,I,84,68.5,72.7,73.3,NaN,93.9,81,88.9,...,95.7,96.4,98.8,103.9,NaN,99.4,96.1,95.6,101.5,NaN
39,2018,L,68.7,77.1,85.4,84.5,NaN,80.8,89,82.6,...,95.3,93.9,94,94.9,NaN,99.2,95.8,92.5,95.2,NaN
41,2018,N,82.3,85,86.3,89.1,NaN,87.1,88.2,91.2,...,98.5,93.8,95,96.2,NaN,96.5,95.8,96,96.6,NaN
42,2018,P,85.2,77.7,81.7,89.4,NaN,86.9,84.5,91.3,...,97.1,95.6,95.9,94.3,NaN,96.7,94,93.1,97.9,NaN
43,2018,R,69.8,80.2,89.2,101.3,NaN,77.4,92.1,93.5,...,97.1,91.8,87.6,92.4,NaN,97.1,91.8,90.4,95.4,NaN
78,2019,G,72.7,77.7,73.9,80.9,78.7,82.8,83.2,82.4,...,95.7,94.3,98.3,93.7,94.8,98.1,95.2,95.2,94.5,92.9
79,2019,S,79.9,73.9,75.8,72.6,74.1,82.3,75,82.4,...,94.7,90.2,96.6,95,94.7,92.9,92.3,98.7,91.7,92.9


In [204]:
# 산업별 기업규모별 기업수(중소기업-소상공인)
# 산업별 기업규모별 종사자수(중소기업-소상공인)

# dataRefine 함수
def dataRefine(refDf):
    temp_df = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    # API Data를 DataFrame형태로 저장할 딕셔너리
    split_dfs = {}
    # Row로 구분되어 있는 조건들(예, '체감', '전망')을 개개의 컬럼으로 만들어주기 위한 준비
    filter_1 = temp_df['ITM_NM'].drop_duplicates().values.tolist()
    # 조건 조합 반복
    for s in filter_1:
        # 'DT'컬럼의 이름을 대체함
        newColName = f"{temp_df['TBL_NM'].unique()[0][:]}_{s}"        
        filtered = temp_df[temp_df['ITM_NM'] == s]
        split_dfs[newColName] = filtered.copy()
        split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
    return split_dfs

def mergeDataFrame(refDf):
    first_df = list(refDf.values())[0]
    df_merged = first_df.iloc[:, [2, 3, 4]].copy()
    for name, sub in list(refDf.items())[1:]:
        sub_unique = sub.drop_duplicates(subset=['PRD_DE', 'C1_NM'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 3, 4]], on=['PRD_DE', 'C1_NM'], how='left')  
    return df_merged


##################### 산업별 기업규모별 기업수(중소기업-소상공인) #####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+&objL1=G+H+I+L+N+P+Q+R+S+&objL2=2+3+4+5+6+7+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=DT_1BD1022'
induCorpCount_Y = mergeDataFrame(dataRefine(urlRequest(url)))
# 산업별 기업규모별 기업수_활동 : actCorpNo
# 산업별 기업규모별 기업수_신생: newCorpNo
# 산업별 기업규모별 기업수_소멸: endCorpNo
induCorpCount_Y.columns = [
    'PRD_DE', 'KSIC',
    'actCorpNo', 'newCorpNo', 'endCorpNo'
]

# 경기 선행지수 만들기
# lag 생성할 대상 컬럼
target_cols = ['actCorpNo', 'newCorpNo', 'endCorpNo']
lag_n = 1  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = induCorpCount_Y.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{lag}Y'
        # REGION 기준으로 그룹별 shift
        lag_series = induCorpCount_Y.groupby('KSIC')[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        induCorpCount_Y.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

#print(econoSBHI_Q['KSIC'].unique())
replace_dict = {
    '도매 및 소매업': 'G', 
    '운수 및 창고업': 'H', 
    '숙박 및 음식점업': 'I', 
    '부동산업': 'L', 
    '사업시설관리 사업지원 및 임대 서비스': 'N',
    '교육서비스업': 'P',
    '보건업 및 사회복지 서비스업': 'Q',
    '예술 스포츠 및 여가관련 서비스업': 'R',
    '협회 및 단체 수리 및 기타 개인서비스업': 'S'
}
induCorpCount_Y['KSIC'] = induCorpCount_Y['KSIC'].replace(replace_dict)
induCorpCount_Y = induCorpCount_Y[induCorpCount_Y['KSIC'].isin(list(replace_dict.values()))]
induCorpCount = induCorpCount_Y.copy()
display(induCorpCount)
save_excel_and_csv(induCorpCount, 'kosis기업수_업종')
####################################################################################


###################### 산업별 기업규모별 종사자수(중소기업-소상공인) ################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+&objL1=G+H+I+L+N+P+Q+R+S+&objL2=5+6+7+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=DT_1BD1023'
induEmployeeCount_Y = mergeDataFrame(dataRefine(urlRequest(url)))
# 산업별 기업규모별 종사자수_활동 : actEmpNo
# 산업별 기업규모별 종사자수_신생: newEmpNo
# 산업별 기업규모별 종사자수_소멸: endEmpNo
induEmployeeCount_Y.columns = [
    'PRD_DE', 'KSIC',
    'actEmpNo', 'newEmpNo', 'endEmpNo'
]

# 경기 선행지수 만들기
# lag 생성할 대상 컬럼
target_cols = ['actEmpNo', 'newEmpNo', 'endEmpNo']
lag_n = 1  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = induEmployeeCount_Y.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{lag}Y'
        # REGION 기준으로 그룹별 shift
        lag_series = induEmployeeCount_Y.groupby('KSIC')[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        induEmployeeCount_Y.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

induEmployeeCount_Y['KSIC'] = induEmployeeCount_Y['KSIC'].replace(replace_dict)
induEmployeeCount_Y = induEmployeeCount_Y[induEmployeeCount_Y['KSIC'].isin(list(replace_dict.values()))]
induEmployeeCount = induEmployeeCount_Y.copy()
display(induEmployeeCount)
save_excel_and_csv(induEmployeeCount, 'kosis종업사수_업종')
####################################################################################


,PRD_DE,KSIC,actCorpNo,actCorpNo_1Y,newCorpNo,newCorpNo_1Y,endCorpNo,endCorpNo_1Y
0,2017,G,186,NaN,4,NaN,3,NaN
1,2018,G,193,186,4,4,-,3
2,2019,G,192,193,2,4,-,-
3,2020,G,200,192,6,2,1,-
4,2021,G,212,200,10,6,2,1
...,...,...,...,...,...,...,...,...
319,2018,S,275498,264693,-,-,-,-
320,2019,S,295356,275498,2,-,-,-
321,2020,S,308196,295356,1,2,-,-
322,2021,S,318346,308196,1,1,0,-


,PRD_DE,KSIC,actEmpNo,actEmpNo_1Y,newEmpNo,newEmpNo_1Y,endEmpNo,endEmpNo_1Y
0,2017,G,2973,NaN,281,NaN,215,NaN
1,2018,G,3061,2973,262,281,217,215
2,2019,G,3119,3061,274,262,212,217
3,2020,G,3216,3119,295,274,217,212
4,2021,G,3317,3216,300,295,216,217
...,...,...,...,...,...,...,...,...
157,2018,S,324,310,58,54,40,41
158,2019,S,346,324,64,58,44,40
159,2020,S,367,346,57,64,44,44
160,2021,S,372,367,58,57,41,44


In [205]:
# 시도/산업중분류별/조직형태별 사업체 및 종사자수
# 시도/산업중분류별/사업장점유형태별 사업체수

# dataRefine 함수
def dataRefine(refDf, text):
    if text == 'pre2020':
        tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'C2_NM', 'C3_NM', 'DT']].copy()
        filter_1 = tempDf['C3_NM'].drop_duplicates().values.tolist()
        filter_2 = tempDf['ITM_NM'].drop_duplicates().values.tolist()
        split_dfs = {}    
        for s in filter_1:
            for p in filter_2:
                # 'DT'컬럼의 이름을 대체함
                newColName = f"{tempDf['TBL_NM'].unique()[0][:]}_{p}_{s}"  
                filtered = tempDf[(tempDf['C3_NM'] == s) & (tempDf['ITM_NM'] == p)]
                split_dfs[newColName] = filtered.copy()
                split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
    else:
        tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'C2_NM', 'DT']].copy()
        filter_1 = tempDf['ITM_NM'].drop_duplicates().values.tolist()
        split_dfs = {}
        for s in filter_1:
            newColName = f"{tempDf['TBL_NM'].unique()[0][:]}_{s}"     
            filtered = tempDf[tempDf['ITM_NM'] == s]
            split_dfs[newColName] = filtered.copy()
            split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
    return split_dfs
 
def mergeDataFrame(refDf, text):
    first_df = list(refDf.values())[0]
    if text == 'pre2020':
        df_merged = first_df.iloc[:, [2, 3, 4, 6]].copy()
        for name, sub in list(refDf.items())[1:]:
            sub_unique = sub.drop_duplicates(subset=['PRD_DE', 'C1_NM', 'C2_NM'])
            df_merged = df_merged.merge(sub_unique.iloc[:, [2, 3, 4, 6]], on=['PRD_DE', 'C1_NM', 'C2_NM'], how='left')
    else:
        df_merged = first_df.iloc[:, [2, 3, 4, 5]].copy()
        for name, sub in list(refDf.items())[1:]:
            sub_unique = sub.drop_duplicates(subset=['PRD_DE', 'C1_NM', 'C2_NM'])
            df_merged = df_merged.merge(sub_unique.iloc[:, [2, 3, 4, 5]], on=['PRD_DE', 'C1_NM', 'C2_NM'], how='left')
    return df_merged


########## 시도/산업중분류별/조직형태별 사업체 및 종사자수 ##########
# 2020년 이전
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=ALL&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe=2019&orgId=142&tblId=DT_1ME0101' 
localInduCorpEmployeeCount_pre2020 = mergeDataFrame(dataRefine(urlRequest(url), 'pre2020'), 'pre2020')

# 2020년 이후
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+T04+T05+T06+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2020&endPrdDe={endYYYY}&orgId=142&tblId=DT_2ME0101'
localInduCorpEmployeeCount_post2020 = mergeDataFrame(dataRefine(urlRequest(url), 'post2020'), 'post2020')

# 두 개의 dataframe을 합침
# 컬럼명이 다르므로 'localInduCorpEmployeeCount_post2020' 컬럼명으로 통일 시키고 concat로 합침
localInduCorpEmployeeCount_pre2020.columns = localInduCorpEmployeeCount_post2020.columns
localInduCorpEmployeeCount_Y = pd.concat([localInduCorpEmployeeCount_pre2020, localInduCorpEmployeeCount_post2020], ignore_index=True)

# 시도/산업중분류별/조직형태별 사업체 및 종사자수_사업체수_합계 : totCompCnt
# 시도/산업중분류별/조직형태별 사업체 및 종사자수_종사자수_합계 : totEmpCnt
# 시도/산업중분류별/조직형태별 사업체 및 종사자수_사업체수_개인 : indiCompCnt
# 시도/산업중분류별/조직형태별 사업체 및 종사자수_종사자수_개인 : indiEmpCnt
# 시도/산업중분류별/조직형태별 사업체 및 종사자수_사업체수_법인 : corpCompCnt	
# 시도/산업중분류별/조직형태별 사업체 및 종사자수_종사자수_법인 : corpEmpCnt
localInduCorpEmployeeCount_Y.columns = [
    'PRD_DE', 'REGION', 'KSIC',
    'totCompCnt', 'totEmpCnt', 'indiCompCnt', 'indiEmpCnt', 'corpCompCnt', 'corpEmpCnt'
]

# 경기 선행지수 만들기
# lag 생성할 대상 컬럼
target_cols = ['totCompCnt', 'totEmpCnt', 'indiCompCnt', 'indiEmpCnt', 'corpCompCnt', 'corpEmpCnt']
lag_n = 1  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = localInduCorpEmployeeCount_Y.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{lag}Y'
        # REGION 기준으로 그룹별 shift
        lag_series = localInduCorpEmployeeCount_Y.groupby(['REGION', 'KSIC'])[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        localInduCorpEmployeeCount_Y.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

#print(econoSBHI_Q['KSIC'].unique())
replace_dict = {
    '도매 및 소매업': 'G', 
    '운수 및 창고업': 'H', 
    '숙박 및 음식점업': 'I', 
    '부동산업': 'L', 
    '사업시설관리 사업지원 및 임대 서비스': 'N',
    '임대업; 부동산 제외' : 'N76',
    '교육서비스업': 'P',
    '보건업 및 사회복지 서비스업': 'Q',
    '예술 스포츠 및 여가관련 서비스업': 'R',
    '수리 및 기타 개인서비스업': 'S'
}
localInduCorpEmployeeCount_Y['KSIC'] = localInduCorpEmployeeCount_Y['KSIC'].replace(replace_dict)
localInduCorpEmployeeCount_Y = localInduCorpEmployeeCount_Y[localInduCorpEmployeeCount_Y['KSIC'].isin(list(replace_dict.values()))]
localInduCorpEmployeeCount = localInduCorpEmployeeCount_Y.copy()
display(localInduCorpEmployeeCount)
save_excel_and_csv(localInduCorpEmployeeCount, 'kosis조직형태별사업체수종사자수_지역_업종')
#########################################################################



# ########## 시도/산업중분류별/사업장점유형태별 사업체수 ##########
# 2020년 이전
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=ALL&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe=2019&orgId=142&tblId=DT_1ME0303'
localInduCorpCount_pre2020 = mergeDataFrame(dataRefine(urlRequest(url), 'pre2020'), 'pre2020')

# 2020년 이후
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2020&endPrdDe={endYYYY}&orgId=142&tblId=DT_2ME0303'
localInduCorpCount_post2020 = mergeDataFrame(dataRefine(urlRequest(url), 'post2020'), 'post2020')

# 두 개의 dataframe을 합침
# 컬럼명이 다르므로 'localInduCorpCount_post2020' 컬럼명으로 통일 시키고 concat로 합침
localInduCorpCount_pre2020.columns = localInduCorpCount_post2020.columns
localInduCorpCount_Y = pd.concat([localInduCorpCount_pre2020, localInduCorpCount_post2020], ignore_index=True)

# 시도/산업중분류별/사업장점유형태별 사업체수_사업체 수 : tCompCnt
# 시도/산업중분류별/사업장점유형태별 사업체수_소유 : ownCompCnt	
# 시도/산업중분류별/사업장점유형태별 사업체수_임차(전대차 포함) : LentCompCnt
localInduCorpCount_Y.columns = [
    'PRD_DE', 'REGION', 'KSIC',
    'tCompCnt', 'ownCompCnt', 'LentCompCnt'
]

# 경기 선행지수 만들기
# lag 생성할 대상 컬럼
target_cols = ['tCompCnt', 'ownCompCnt', 'LentCompCnt']
lag_n = 1  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = localInduCorpCount_Y.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{lag}Y'
        # REGION 기준으로 그룹별 shift
        lag_series = localInduCorpCount_Y.groupby(['REGION', 'KSIC'])[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        localInduCorpCount_Y.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

#print(econoSBHI_Q['KSIC'].unique())
replace_dict = {
    '도매 및 소매업': 'G', 
    '운수 및 창고업': 'H', 
    '숙박 및 음식점업': 'I', 
    '부동산업': 'L', 
    '사업시설관리 사업지원 및 임대 서비스': 'N',
    '임대업; 부동산 제외' : 'N76',
    '교육서비스업': 'P',
    '보건업 및 사회복지 서비스업': 'Q',
    '예술 스포츠 및 여가관련 서비스업': 'R',
    '수리 및 기타 개인서비스업': 'S'
}
localInduCorpCount_Y['KSIC'] = localInduCorpCount_Y['KSIC'].replace(replace_dict)
localInduCorpCount_Y = localInduCorpCount_Y[localInduCorpCount_Y['KSIC'].isin(list(replace_dict.values()))]
localInduCorpCount = localInduCorpCount_Y.copy()
display(localInduCorpCount)
save_excel_and_csv(localInduCorpCount, 'kosis사업장점유형태별사업체수_지역_업종')


,PRD_DE,REGION,KSIC,totCompCnt,totCompCnt_1Y,totEmpCnt,totEmpCnt_1Y,indiCompCnt,indiCompCnt_1Y,indiEmpCnt,indiEmpCnt_1Y,corpCompCnt,corpCompCnt_1Y,corpEmpCnt,corpEmpCnt_1Y
0,2018,전국,G,890665,NaN,1827061,NaN,813944,NaN,1622282,NaN,76721,NaN,204779,NaN
1,2019,전국,G,891257,890665,1870109,1827061,803402,813944,1634602,1622282,87855,76721,235507,204779
8,2018,전국,I,649702,NaN,1550726,NaN,644365,NaN,1534746,NaN,5337,NaN,15980,NaN
9,2019,전국,I,660126,649702,1552837,1550726,655690,644365,1537075,1534746,4436,5337,15762,15980
14,2018,전국,L,111602,NaN,170846,NaN,96154,NaN,138992,NaN,15448,NaN,31854,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,2021,제주특별자치도,L,2071,2143,2483,2611,1223,1642,1399,1847,848,501,1084,764
687,2022,제주특별자치도,L,2163,2071,2557,2483,1445,1223,1585,1399,718,848,972,1084
694,2020,제주특별자치도,R,1477,1328,2410,2684,1419,1325,2229,2679,58,3,181,5
695,2021,제주특별자치도,R,1535,1477,2527,2410,1480,1419,2393,2229,55,58,134,181


,PRD_DE,REGION,KSIC,tCompCnt,tCompCnt_1Y,ownCompCnt,ownCompCnt_1Y,LentCompCnt,LentCompCnt_1Y
0,2018,전국,G,890665,NaN,189188,NaN,693270,NaN
1,2019,전국,G,891257,890665,182091,189188,709166,693270
8,2018,전국,I,649702,NaN,141074,NaN,507050,NaN
9,2019,전국,I,660126,649702,122469,141074,537657,507050
14,2018,전국,L,111602,NaN,23743,NaN,87034,NaN
15,2019,전국,L,113551,111602,22558,23743,90993,87034
16,2018,전국,L,111602,113551,23743,22558,87034,90993
17,2019,전국,L,113551,111602,22558,23743,90993,87034
24,2018,전국,N76,12178,NaN,2546,NaN,9499,NaN
25,2019,전국,N76,12604,12178,2642,2546,9962,9499


In [206]:
# 시도/산업중분류별/사업체당 보증금 및 월세, 매출대비 지급비율(단위: 만원, %)
# 시도/산업중분류별 영업비용
# 시도/산업중분류별/영업이익규모별 사업체수 비율(개, 백만원, %)
# 시도별 경제활동별 지역내총생산(백만원, %p)
# 시도별 산업중분류별 기업규모별 매출액(소상공인, 억원)

# dataRefine 함수
def dataRefine(refDf):
    tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'C2_NM', 'DT']].copy()
    filter_1 = tempDf['ITM_NM'].drop_duplicates().values.tolist()
    split_dfs = {}
    for s in filter_1:
        newColName = f"{tempDf['TBL_NM'].unique()[0][:]}_{s}"     
        filtered = tempDf[tempDf['ITM_NM'] == s]
        split_dfs[newColName] = filtered.copy()
        split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
    return split_dfs 

def mergeDataFrame(refDf):
    first_df = list(refDf.values())[0]
    df_merged = first_df.iloc[:, [2, 3, 4, 5]].copy()
    for name, sub in list(refDf.items())[1:]:
        sub_unique = sub.drop_duplicates(subset=['PRD_DE', 'C1_NM', 'C2_NM'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 3, 4, 5]], on=['PRD_DE', 'C1_NM', 'C2_NM'], how='left')
    return df_merged



########## 시도/산업중분류별/사업체당 보증금 및 월세, 매출대비 지급비율(단위: 만원, %) ##########
# 2020년 이전
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+T04+T05+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe=2019&orgId=142&tblId=DT_1ME0305'
df = urlRequest(url)
refinedDf = dataRefine(df)
localInduDepositCost_pre2020 = mergeDataFrame(refinedDf)

# 2020년 이후
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+T04+T05+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2020&endPrdDe={endYYYY}&orgId=142&tblId=DT_2ME0305'
df = urlRequest(url)
refinedDf = dataRefine(df)
localInduDepositCost_post2020 = mergeDataFrame(refinedDf)

# 두 개의 dataframe을 합침
# 컬럼명이 다르므로 'localInduCorpCount_post2020' 컬럼명으로 통일 시키고 concat로 합침
localInduDepositCost_pre2020.columns = localInduDepositCost_post2020.columns
localInduDepositCost_Y = pd.concat([localInduDepositCost_pre2020, localInduDepositCost_post2020], ignore_index=True)

# 시도/산업중분류별/사업체당 보증금 및 월세 매출대비 지급비율_보증금 있는 월세_보증금(만원) : deposit
# 시도/산업중분류별/사업체당 보증금 및 월세 매출대비 지급비율_보증금 있는 월세_월세(만원) :monRent	
# 시도/산업중분류별/사업체당 보증금 및 월세 매출대비 지급비율_보증금 없는 월세(만원) : 0monRent	
# 시도/산업중분류별/사업체당 보증금 및 월세 매출대비 지급비율_전세(만원) : ltRent
# 시도/산업중분류별/사업체당 보증금 및 월세 매출대비 지급비율_매출대비 일정비율 지급(%) : rentPayRatio
localInduDepositCost_Y.columns = [
    'PRD_DE', 'REGION', 'KSIC',
    'deposit', 'monRent', '0monRent', 'ltRent', 'rentPayRatio'
]

# 경기 선행지수 만들기
# lag 생성할 대상 컬럼
target_cols = ['deposit', 'monRent', '0monRent', 'ltRent', 'rentPayRatio']
lag_n = 1  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = localInduDepositCost_Y.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{lag}Y'
        # REGION 기준으로 그룹별 shift
        lag_series = localInduDepositCost_Y.groupby(['REGION', 'KSIC'])[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        localInduDepositCost_Y.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

replace_dict = {
    '도매 및 소매업' : 'G',
    '운수 및 창고업': 'H', 
    '숙박 및 음식점업' : 'I',
    '부동산업' : 'L',
    '사업시설관리 사업지원 및 임대 서비스업' : 'N',
    '사업지원 서비스업' : 'N75',
    '임대업; 부동산 제외' : 'N76',
    '교육 서비스업' : 'P',
    '예술 스포츠 및 여가관련 서비스업' : 'R',
    '수리 및 기타 개인서비스업' : 'S'
}
localInduDepositCost_Y['KSIC'] = localInduDepositCost_Y['KSIC'].replace(replace_dict)
localInduDepositCost_Y = localInduDepositCost_Y[localInduDepositCost_Y['KSIC'].isin(list(replace_dict.values()))]
localInduDepositCost = localInduDepositCost_Y.copy()
display(localInduDepositCost)
save_excel_and_csv(localInduDepositCost, 'kosis보증금및월세_매출대비지급비율_지역_업종')
##########################################################################################


########## 시도/산업중분류별 영업비용 ####################
# 2020년 이전
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=00+10+50+20+30+40+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe=2019&orgId=142&tblId=DT_1ME0311'
localInduOpCost_pre2020 = mergeDataFrame(dataRefine(urlRequest(url)))

# 2020년 이후
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T06+T03+T04+T05+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2020&endPrdDe={endYYYY}&orgId=142&tblId=DT_2ME0311'
localInduOpCost_post2020 = mergeDataFrame(dataRefine(urlRequest(url)))

# 두 개의 dataframe을 합침
# 컬럼명이 다르므로 'localInduCorpCount_post2020' 컬럼명으로 통일 시키고 concat로 합침
localInduOpCost_pre2020.columns = localInduOpCost_post2020.columns
localInduOpCost_Y = pd.concat([localInduOpCost_pre2020, localInduOpCost_post2020], ignore_index=True)

# 시도/산업중분류별/영업비용_사업체 수 : opCompCnt
# 시도/산업중분류별/영업비용_영업비용 : opOpPay
# 시도/산업중분류별/영업비용_영업비용_급여총액 : opSalaryPay
# 시도/산업중분류별/영업비용_영업비용_임차료 : opLentPay
# 시도/산업중분류별/영업비용_영업비용_기타 : opEtcPay
# 시도/산업중분류별/영업비용_영업비용_매출원가 : opCostGoods
localInduOpCost_Y.columns = [
    'PRD_DE', 'REGION', 'KSIC',
    'opCompCnt', 'opOpPay', 'opSalaryPay', 'opLentPay', 'opEtcPay', 'opCostGoods'  
]

# 경기 선행지수 만들기
# lag 생성할 대상 컬럼
target_cols = ['opCompCnt', 'opOpPay', 'opSalaryPay', 'opLentPay', 'opEtcPay', 'opCostGoods']
lag_n = 1  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = localInduOpCost_Y.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{lag}Y'
        # REGION 기준으로 그룹별 shift
        lag_series = localInduOpCost_Y.groupby(['REGION', 'KSIC'])[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        localInduOpCost_Y.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

localInduOpCost_Y['KSIC'] = localInduOpCost_Y['KSIC'].replace(replace_dict)
localInduOpCost_Y = localInduOpCost_Y[localInduOpCost_Y['KSIC'].isin(list(replace_dict.values()))]
localInduOpCost = localInduOpCost_Y.copy()
display(localInduOpCost)
save_excel_and_csv(localInduDepositCost, 'kosis영업비용_지역_업종')
##########################################################################################


########## 시도/산업중분류별/영업이익규모별 사업체수 비율(개, 백만원, %) ####################
# 2020년 이전
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=00+10+20+30+40+50+60+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2018&endPrdDe=2020&orgId=142&tblId=DT_1ME0312'
localInduOpprofitCorpCount_pre2020 = mergeDataFrame(dataRefine(urlRequest(url)))

# 2020년 이후
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+T04+T05+T06+T07+&objL1=ALL&objL2=G+G45+G46+G47+I+I55+I56+L+L68+N+N74+N75+N76+P+P85+R+R90+R91+S+S95+S96+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe=2020&endPrdDe=2022&orgId=142&tblId=DT_2ME0312'
localInduOpprofitCorpCount_post2020 = mergeDataFrame(dataRefine(urlRequest(url)))

# 두 개의 dataframe을 합침
# 컬럼명이 다르므로 'localInduOpprofitCorpCount_post2020' 컬럼명으로 통일 시키고 concat로 합침
localInduOpprofitCorpCount_pre2020.columns = localInduOpprofitCorpCount_post2020.columns
localInduOpprofitCorpCount_Y = pd.concat([localInduOpprofitCorpCount_pre2020, localInduOpprofitCorpCount_post2020], ignore_index=True)

# 시도/산업중분류별/영업이익규모별 사업체수 비율_사업체 수	: opSizeCompCnt
# 시도/산업중분류별/영업이익규모별 사업체수 비율_영업이익	: opAmt
# 시도/산업중분류별/영업이익규모별 사업체수 비율_비율합계(%) : opRatioTot	
# 시도/산업중분류별/영업이익규모별 사업체수 비율_1천만원미만(%) : opRatio_1	
# 시도/산업중분류별/영업이익규모별 사업체수 비율_1천만원~3천만원미만(%) : opRatio_1_3	
# 시도/산업중분류별/영업이익규모별 사업체수 비율_3천만원~5천만원미만(%) : opRatio_3_5	
# 시도/산업중분류별/영업이익규모별 사업체수 비율_5천만원 이상(%) : opRatio_5_
localInduOpprofitCorpCount_Y.columns = [
    'PRD_DE', 'REGION', 'KSIC',
    'opSizeCompCnt', 'opAmt', 'opRatioTot', 'opRatio_1', 'opRatio_1_3', 'opRatio_3_5' , 'opRatio_5_'
]

# 경기 선행지수 만들기
# lag 생성할 대상 컬럼
target_cols = ['opSizeCompCnt', 'opAmt', 'opRatioTot', 'opRatio_1', 'opRatio_1_3', 'opRatio_3_5' , 'opRatio_5_']
lag_n = 1  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = localInduOpprofitCorpCount_Y.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{lag}Y'
        # REGION 기준으로 그룹별 shift
        lag_series = localInduOpprofitCorpCount_Y.groupby(['REGION', 'KSIC'])[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        localInduOpprofitCorpCount_Y.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

localInduOpprofitCorpCount_Y['KSIC'] = localInduOpprofitCorpCount_Y['KSIC'].replace(replace_dict)
localInduOpprofitCorpCount_Y = localInduOpprofitCorpCount_Y[localInduOpprofitCorpCount_Y['KSIC'].isin(list(replace_dict.values()))]
localInduOpprofitCorpCount = localInduOpprofitCorpCount_Y.copy()
display(localInduOpprofitCorpCount)
save_excel_and_csv(localInduOpprofitCorpCount, 'kosis영업이익규모별사업체수_지역_업종')
##########################################################################################


#################### 시도별 경제활동별 지역내총생산(백만원, %p) #################### 
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T1+T2+T3+&objL1=ALL&objL2=SER+G00+H00+I00+L00+N00+O00+P00+Q00+R00+S00+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=DT_1C91'
localInduGDP_Y = mergeDataFrame(dataRefine(urlRequest(url)))

# 시도별 경제활동별 지역내총생산_명목 : nominalGDP	
# 시도별 경제활동별 지역내총생산_실질 : realGDP	
# 시도별 경제활동별 지역내총생산_실질기여도 : GDPContrib
localInduGDP_Y.columns = [
    'PRD_DE', 'REGION', 'KSIC',
    'nominalGDP', 'realGDP', 'GDPContrib'
]

# 경기 선행지수 만들기
# lag 생성할 대상 컬럼
target_cols = ['nominalGDP', 'realGDP', 'GDPContrib']
lag_n = 1  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = localInduGDP_Y.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{lag}Y'
        # REGION 기준으로 그룹별 shift
        lag_series = localInduGDP_Y.groupby(['REGION', 'KSIC'])[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        localInduGDP_Y.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

localInduGDP_Y['KSIC'] = localInduGDP_Y['KSIC'].replace(replace_dict)
localInduGDP_Y = localInduGDP_Y[localInduGDP_Y['KSIC'].isin(list(replace_dict.values()))]
localInduGDP = localInduGDP_Y.copy()
display(localInduGDP)
save_excel_and_csv(localInduGDP, 'kosis지역내총생산_지역_업종')
##########################################################################################


#################### 시도별 산업중분류별 기업규모별 매출액(소상공인, 억원) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=IM_G+IM_H+IM_I+IM_L+IM_N+IM_P+IM_Q+IM_R+IM_S+&objL2=ALL&objL3=16142T2524+&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=142&tblId=DT_BR_C001'
localInduRevenue_Y = mergeDataFrame(dataRefine(urlRequest(url)))

localInduRevenue_Y.columns = [
    'PRD_DE', 'KSIC', 'REGION', 'revenue'
]

# 경기 선행지수 만들기
# lag 생성할 대상 컬럼
target_cols = ['revenue']
lag_n = 1  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = localInduRevenue_Y.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{lag}Y'
        # REGION 기준으로 그룹별 shift
        lag_series = localInduRevenue_Y.groupby(['REGION', 'KSIC'])[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        localInduRevenue_Y.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

localInduRevenue_Y['KSIC'] = localInduRevenue_Y['KSIC'].str[0]
localInduRevenue = localInduRevenue_Y.copy()
display(localInduRevenue)
save_excel_and_csv(localInduRevenue, 'kosis매출액_지역_업종')
##########################################################################################

,PRD_DE,REGION,KSIC,deposit,deposit_1Y,monRent,monRent_1Y,0monRent,0monRent_1Y,ltRent,ltRent_1Y,rentPayRatio,rentPayRatio_1Y
0,2018,전국,G,2482,NaN,132,NaN,70,NaN,4338,NaN,21.2,NaN
1,2019,전국,G,2651,2482,138,132,79,70,4504,4338,18.7,21.2
8,2018,전국,I,2331,NaN,129,NaN,71,NaN,1161,NaN,16,NaN
9,2019,전국,I,2410,2331,134,129,105,71,2112,1161,16,16
14,2018,전국,L,1992,NaN,93,NaN,63,NaN,3644,NaN,47.4,NaN
15,2019,전국,L,1902,1992,96,93,54,63,4528,3644,21.8,47.4
16,2018,전국,L,1992,1902,93,96,63,54,3644,4528,47.4,21.8
17,2019,전국,L,1902,1992,96,93,54,63,4528,3644,21.8,47.4
18,2018,전국,N,1488,NaN,84,NaN,58,NaN,2905,NaN,25.2,NaN
19,2019,전국,N,1658,1488,87,84,68,58,3936,2905,13.1,25.2


,PRD_DE,REGION,KSIC,opCompCnt,opCompCnt_1Y,opOpPay,opOpPay_1Y,opSalaryPay,opSalaryPay_1Y,opLentPay,opLentPay_1Y,opEtcPay,opEtcPay_1Y,opCostGoods,opCostGoods_1Y
0,2018,전국,G,890665,NaN,265231259,NaN,21055921,NaN,13417319,NaN,230758018,NaN,NaN,NaN
1,2019,전국,G,891257,890665,271182966,265231259,21953961,21055921,13885283,13417319,21294860,230758018,214048861,NaN
8,2018,전국,I,649702,NaN,57578663,NaN,10092934,NaN,7713494,NaN,39772235,NaN,NaN,NaN
9,2019,전국,I,660126,649702,60841411,57578663,10979140,10092934,7831166,7713494,6640946,39772235,35390160,NaN
14,2018,전국,L,111602,NaN,6045623,NaN,1518907,NaN,976941,NaN,3549775,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,2021,제주특별자치도,P,2798,2528,111046,106137,27964,29604,23209,20938,35670,48035,24203,7561
1118,2022,제주특별자치도,P,2791,2798,111732,111046,35704,27964,27358,23209,30065,35670,18606,24203
1119,2020,제주특별자치도,R,1477,1328,74033,55592,11104,12585,15566,14348,34832,11055,12530,17603
1120,2021,제주특별자치도,R,1535,1477,78733,74033,15274,11104,19041,15566,28026,34832,16393,12530


,PRD_DE,REGION,KSIC,opSizeCompCnt,opSizeCompCnt_1Y,opAmt,opAmt_1Y,opRatioTot,opRatioTot_1Y,opRatio_1,opRatio_1_1Y,opRatio_1_3,opRatio_1_3_1Y,opRatio_3_5,opRatio_3_5_1Y,opRatio_5_,opRatio_5__1Y
0,2018,전국,G,890665,NaN,36218057,NaN,100,NaN,16.9,NaN,32.7,NaN,24.1,NaN,26.3,NaN
1,2019,전국,G,891257,890665,35245359,36218057,100,100,18.9,16.9,32.2,32.7,21.8,24.1,27.1,26.3
2,2020,전국,G,911339,891257,18964436,35245359,100,100,45.7,18.9,31.4,32.2,10.8,21.8,12.1,27.1
12,2018,전국,I,649702,NaN,20391265,NaN,100,NaN,10.9,NaN,42.4,NaN,26.2,NaN,20.6,NaN
13,2019,전국,I,660126,649702,19041204,20391265,100,100,12.7,10.9,39.7,42.4,26.5,26.2,21.2,20.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1342,2021,제주특별자치도,P,2798,2528,45035,28727,100,100,32.5,40.4,43,58,19.6,1.6,4.9,0
1343,2022,제주특별자치도,P,2791,2798,50777,45035,100,100,34,32.5,47.1,43,11.5,19.6,7.4,4.9
1344,2020,제주특별자치도,R,1477,1358,17919,22136,100,100,61.3,60.2,24.5,24,11.6,12.9,2.6,3
1345,2021,제주특별자치도,R,1535,1477,17032,17919,100,100,44.8,61.3,37.1,24.5,11.1,11.6,7,2.6


,PRD_DE,REGION,KSIC,nominalGDP,nominalGDP_1Y,realGDP,realGDP_1Y,GDPContrib,GDPContrib_1Y
0,2017,전국,G,171888528,NaN,164666777,NaN,0.25,NaN
1,2018,전국,G,175881303,171888528,169194046,164666777,0.24,0.25
2,2019,전국,G,178189719,175881303,175673015,169194046,0.34,0.24
3,2020,전국,G,175029822,178189719,175029822,175673015,-0.03,0.34
4,2021,전국,G,185232614,175029822,177655406,175029822,0.13,-0.03
...,...,...,...,...,...,...,...,...,...
1159,2018,제주특별자치도,P,1180700,1106869,1228853,1173072,0.24,0.19
1160,2019,제주특별자치도,P,1246160,1180700,1254676,1228853,0.11,0.24
1161,2020,제주특별자치도,P,1217278,1246160,1217278,1254676,-0.17,0.11
1162,2021,제주특별자치도,P,1319996,1217278,1299219,1217278,0.38,-0.17


,PRD_DE,KSIC,REGION,revenue,revenue_1Y
0,2019,G,전국,3932001,NaN
1,2020,G,전국,4015885,3932001
2,2021,G,전국,4400729,4015885
3,2022,G,전국,4667061,4400729
4,2019,G,서울,926650,NaN
...,...,...,...,...,...
643,2022,S,제주,2654,2385
644,2019,S,세종,818,NaN
645,2020,S,세종,868,818
646,2021,S,세종,961,868


In [207]:
# 시도별 지역내총생산에 대한 지출
# 시도별 1인당 지역내총생산, 지역총소득, 개인소득

# dataRefine 함수
def dataRefine(refDf):
    temp_df = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    split_dfs = {}
    filter_1 = temp_df['ITM_NM'].drop_duplicates().values.tolist()
    for s in filter_1:
        # 'DT'컬럼의 이름을 대체함
        newColName = f"{temp_df['TBL_NM'].unique()[0][:]}_{s}"        
        filtered = temp_df[temp_df['ITM_NM'] == s]
        split_dfs[newColName] = filtered.copy()
        split_dfs[newColName].rename(columns={'DT': newColName}, inplace=True)
    return split_dfs

def mergeDataFrame(refDf):
    first_df = list(refDf.values())[0]
    df_merged = first_df.iloc[:, [2, 3, 4]].copy()
    for name, sub in list(refDf.items())[1:]:
        sub_unique = sub.drop_duplicates(subset=['PRD_DE', 'C1_NM'])
        df_merged = df_merged.merge(sub_unique.iloc[:, [2, 3, 4]], on=['PRD_DE', 'C1_NM'], how='left')  
    return df_merged

#################### 시도별 지역내총생산에 대한 지출 ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T01+T02+T03+&objL1=ALL&objL2=E00+&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=DT_1C93'
localInduGDPExpense = mergeDataFrame(dataRefine(urlRequest(url)))
display(localInduGDPExpense)
save_excel_and_csv(localInduGDPExpense, 'localInduGDPExpense')
#########################################################################


#################### 시도별 1인당 지역내총생산, 지역총소득, 개인소득 #################### 
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T1+T2+T3+T4+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Y&startPrdDe={startYYYY}&endPrdDe={endYYYY}&orgId=101&tblId=DT_1C96'
localGDPperPerson = mergeDataFrame(dataRefine(urlRequest(url)))
display(localGDPperPerson)
save_excel_and_csv(localGDPperPerson, 'localGDPperPerson')
#########################################################################

,PRD_DE,C1_NM,시도별 지역내총생산에 대한 지출_명목,시도별 지역내총생산에 대한 지출_실질,시도별 지역내총생산에 대한 지출_실질기여도
0,2017,전국,1935809579,1965889849,3.4050759841
1,2018,전국,2007811627,2026457077,3.0809064924
2,2019,전국,2042981250,2075340783,2.4122744559
3,2020,전국,2062489906,2062489906,-0.6192176751
4,2021,전국,2224179729,2155993359,4.5335229389
...,...,...,...,...,...
103,2018,제주특별자치도,22040507,22777044,0.9435497133
104,2019,제주특별자치도,22267086,22951902,0.7676955889
105,2020,제주특별자치도,21668785,21668785,-5.5904620838
106,2021,제주특별자치도,22757094,22346390,3.127101958


,PRD_DE,C1_NM,시도별 1인당 지역내총생산 지역총소득 개인소득_1인당 지역내총생산,시도별 1인당 지역내총생산 지역총소득 개인소득_1인당 지역총소득,시도별 1인당 지역내총생산 지역총소득 개인소득_1인당 개인소득,시도별 1인당 지역내총생산 지역총소득 개인소득_1인당 민간소비
0,2017,전국,37690,37835,20309,18477
1,2018,전국,38922,39070,21288,19277
2,2019,전국,39467,39787,22289,19778
3,2020,전국,39789,40115,23042,19083
4,2021,전국,42963,43415,23726,20292
...,...,...,...,...,...,...
103,2018,제주특별자치도,33673,32497,19402,19144
104,2019,제주특별자치도,33545,32250,19979,19741
105,2020,제주특별자치도,32404,31825,20835,18988
106,2021,제주특별자치도,33861,36153,21396,19966


In [208]:
# 상권별 집합상가 임대가격지수(단위: 2024.2Q=100.0)
# 상권별 통합상가 임대가격지수(시계열이 짧음, 2024 1Q~)
# 상권별 소규모상가 임대가격지수(단위: 2024.2Q=100.0)
# 상권별 오피스 임대가격지수(단위: 2024.2Q=100.0)
# 상권별 중대형상가 임대가격지수(단위: 2024.2Q=100.0)


# 추출 데이터가 월단위 데이터이므로, 이를 Quater 단위로 변환
# 월 → 분기(Q1~Q4)로 변환하는 함수 : 3,6,9,12월 자료만 Quater로 변환하고 나머지는 Unknown 처리
# 원천데이터가 Q1~Q4가 아니라 '01~04'로 표기되어 있음
def month_to_quarter(prd):
    year = prd[:4]
    month = prd[-2:]
    if month in ['01']:
        quarter = 'Q1'
    elif month in ['02']:
        quarter = 'Q2'
    elif month in ['03']:
        quarter = 'Q3'
    elif month in ['04']:
        quarter = 'Q4'
    else:
        quarter = 'Unknown'
    return f"{year}{quarter}"

# dataRefine 함수
def dataRefine(refDf):
    tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    newColName = tempDf['TBL_NM'].unique()[0]          # 'DT' 컬럼 네임 교체
    tempDf.rename(columns={'DT': newColName}, inplace=True)
    tempDf['PRD_DE'] = tempDf['PRD_DE'].apply(month_to_quarter)
    return tempDf[~tempDf['PRD_DE'].str.contains('Unknown')]
    
def mergeTimeSeriesDataFrame(refDf):
    df = initQuaterDf()
    return df.merge(refDf, on='PRD_DE', how='left') 



#################### 상권별 집합상가 임대가격지수(단위: 2024.2Q=100.0) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=A01+A02+A03+A04+A05+A06+A07+A08+A09+A10+A11+A12+A13+A14+A15+A16+A17+A18+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=408&tblId=DT_40801_N3201_06'
refinedDf = dataRefine(urlRequest(url))
collectiveShopDistrictRentIndex = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
##########################################################################################


#################### 상권별 통합상가 임대가격지수(2024 1Q부터만 데이터가 있음) #################### 
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=ALL&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202401&endPrdDe=202501&orgId=408&tblId=DT_40801_N5201_06'
refinedDf = dataRefine(urlRequest(url))
integratedShopDistrictRentIndex = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
##################################################################################################


#################### 상권별 소규모상가 임대가격지수(단위: 2024.2Q=100.0) #################### 
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=A01+A02+A03+A04+A05+A06+A07+A08+A09+A10+A11+A12+A13+A14+A15+A16+A17+A18+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=408&tblId=DT_40801_N4201_06'
refinedDf = dataRefine(urlRequest(url))
smallShopDistrictRentIndex = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
##########################################################################################


#################### 상권별 오피스 임대가격지수(단위: 2024.2Q=100.0) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=A01+A02+A03+A04+A05+A06+A07+A08+A10+A11+A12+A13+A14+A15+A16+A17+A18+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=408&tblId=DT_40801_N1201_06'
refinedDf = dataRefine(urlRequest(url))
officeShopDistrictRentIndex = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
##########################################################################################


#################### 상권별 중대형상가 임대가격지수(단위: 2024.2Q=100.0) ####################
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=T001+&objL1=A01+A02+A03+A04+A05+A06+A07+A08+A09+A10+A11+A12+A13+A14+A15+A16+A17+A18+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe={startYYYYMM}&endPrdDe={endYYYYMM}&orgId=408&tblId=DT_40801_N2201_06'
refinedDf = dataRefine(urlRequest(url))
middleBigShopDistrictRentIndex = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
##########################################################################################

# Dataframe을 결합하여 상권별 임대가격지수 Dataframe을 만듦 (상권별 통합상가 임대가격지수(2024 1Q부터만 데이터가 있음)는 결합대상 제외)
ShopDistrictRentIndex_Q = collectiveShopDistrictRentIndex.merge(smallShopDistrictRentIndex, on=['PRD_DE', 'C1_NM'], how='left') \
               .merge(officeShopDistrictRentIndex, on=['PRD_DE', 'C1_NM'], how='left') \
               .merge(middleBigShopDistrictRentIndex, on=['PRD_DE', 'C1_NM'], how='left')

# 상권별 집합 상가 임대가격지수 : collecShopRentIdx	
# 상권별 소규모 상가 임대가격지수 : smallShopRentIdx
# 상권별 오피스 임대가격지수 : officeShopRentIdx
# 상권별 중대형 상가 임대가격지수 : midBigShopRentIdx
ShopDistrictRentIndex_Q.columns = [
    'PRD_DE', 'REGION', 
    'collecShopRentIdx', 'smallShopRentIdx', 'officeShopRentIdx', 'midBigShopRentIdx'
]
display(ShopDistrictRentIndex_Q)

# lag 생성할 대상 컬럼
target_cols = ['collecShopRentIdx', 'smallShopRentIdx', 'officeShopRentIdx', 'midBigShopRentIdx']
lag_n = 4  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = ShopDistrictRentIndex_Q.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{3*lag}M'
        # REGION 기준으로 그룹별 shift
        lag_series = ShopDistrictRentIndex_Q.groupby('REGION')[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        ShopDistrictRentIndex_Q.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

# 'Q4'로 끝나는 row만 필터링
df_q4 = ShopDistrictRentIndex_Q[ShopDistrictRentIndex_Q['PRD_DE'].str.endswith('Q4')].copy()
df_q4['PRD_DE'] = df_q4['PRD_DE'].str.replace('Q4', '', regex=False)
ShopDistrictRentIndex = df_q4.copy()

display(ShopDistrictRentIndex)
save_excel_and_csv(ShopDistrictRentIndex, 'kosis상가오피스임대가격지수_지역')

,PRD_DE,REGION,collecShopRentIdx,smallShopRentIdx,officeShopRentIdx,midBigShopRentIdx
0,2018Q1,전국,104.9027925748,106.4770839558,99.4575170344,104.9779512231
1,2018Q1,서울,101.6782698735,101.631843564,96.8386807594,100.345987702
2,2018Q1,부산,106.5378163003,106.8450785492,110.2405776648,106.6294403863
3,2018Q1,대구,107.6678524211,106.3837850553,104.0552743584,106.3589619076
4,2018Q1,인천,103.6875732383,106.0021004897,105.5112599134,104.2718987913
...,...,...,...,...,...,...
355,2022Q4,전북,102.2162019813,102.2948654595,100.5416611915,102.0838261674
356,2022Q4,전남,101.926831868,101.532991943,100.8299198982,100.9803470089
357,2022Q4,경북,100.3270092161,100.6063540969,100.7586398884,100.7889957506
358,2022Q4,경남,100.5155464566,101.6805429512,100.7185199024,102.1287583259


,PRD_DE,REGION,collecShopRentIdx,collecShopRentIdx_3M,collecShopRentIdx_6M,collecShopRentIdx_9M,collecShopRentIdx_12M,smallShopRentIdx,smallShopRentIdx_3M,smallShopRentIdx_6M,...,officeShopRentIdx,officeShopRentIdx_3M,officeShopRentIdx_6M,officeShopRentIdx_9M,officeShopRentIdx_12M,midBigShopRentIdx,midBigShopRentIdx_3M,midBigShopRentIdx_6M,midBigShopRentIdx_9M,midBigShopRentIdx_12M
54,2018,전국,104.5435644539,104.7118824432,104.8497744863,104.9027925748,NaN,105.9143717705,106.2971696327,106.4948276553,...,98.9510079623,99.0922657474,99.2980966157,99.4575170344,NaN,104.6803764941,104.9010485451,104.9547169524,104.9779512231,NaN
55,2018,서울,101.6908719412,101.7142177875,101.6779740525,101.6782698735,NaN,102.0418805962,102.0870899078,101.9267425297,...,96.4083088499,96.5713375058,96.6627243893,96.8386807594,NaN,100.7788908761,100.7746313706,100.6002575175,100.345987702,NaN
56,2018,부산,105.7820790481,106.2831237756,106.4353086288,106.5378163003,NaN,106.020222304,106.6696875961,107.1857989922,...,108.4587778936,109.2055524352,110.0853625107,110.2405776648,NaN,106.1005992782,106.5745863162,106.9756758477,106.6294403863,NaN
57,2018,대구,107.7318511549,107.7647789713,107.7581528799,107.6678524211,NaN,106.1232130191,106.3408387877,106.5340749063,...,103.2660174968,103.3754414837,104.0552743584,104.0552743584,NaN,106.299373113,106.4135558607,106.4409131094,106.3589619076,NaN
58,2018,인천,104.2051140868,104.1078380096,104.0515343681,103.6875732383,NaN,106.182687028,106.1325610947,106.1833244305,...,105.7904048311,105.5194422784,105.5743069109,105.5112599134,NaN,104.4059692917,104.3880816258,104.5901394754,104.2718987913,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2022,전북,102.2162019813,102.4531347169,102.5869051211,102.8448599345,102.8958132054,102.2948654595,102.7688388114,102.9502798476,...,100.5416611915,100.8015868598,100.8015868598,100.8630203369,100.8831869836,102.0838261674,102.4573354199,102.72007503,103.2661273632,103.4033290853
356,2022,전남,101.926831868,102.1686613242,103.0840571114,103.1195070231,103.4614865651,101.532991943,101.6720334095,101.9306695519,...,100.8299198982,100.9081648142,101.1464610553,101.2354983736,101.7315441606,100.9803470089,101.1075074203,101.3286014816,101.3631349538,101.5683087613
357,2022,경북,100.3270092161,100.509971487,100.5562004631,100.6064885299,100.6735572207,100.6063540969,101.0106275017,101.0351633235,...,100.7586398884,101.0387985665,101.2337992025,101.0947626377,101.1837460392,100.7889957506,101.121862631,101.3098076302,101.3990861409,101.4996709424
358,2022,경남,100.5155464566,100.7104242525,100.7282074529,100.4879413159,100.661937793,101.6805429512,101.9017747551,102.0257109887,...,100.7185199024,100.8434726578,101.080094282,100.9305902659,101.0078243909,102.1287583259,102.4801845285,102.5765825004,102.2400869741,102.3607404502


In [209]:
# 상권별 소규모 상가 공실률(단위 : %)
# 상권별 오피스 공실률 (단위 : %)
# 상권별 중대형 상가 공실률(단위 : %)
# 상권별 집합 상가 공실률 (시계열이 22년 4분기 부터밖에 없음)

# 추출 데이터가 월단위 데이터이므로, 이를 Quater 단위로 변환
# 월 → 분기(Q1~Q4)로 변환하는 함수 : 3,6,9,12월 자료만 Quater로 변환하고 나머지는 Unknown 처리
# 원천데이터가 Q1~Q4가 아니라 '01~04'로 표기되어 있음
def month_to_quarter(prd):
    year = prd[:4]
    month = prd[-2:]
    if month in ['01']:
        quarter = 'Q1'
    elif month in ['02']:
        quarter = 'Q2'
    elif month in ['03']:
        quarter = 'Q3'
    elif month in ['04']:
        quarter = 'Q4'
    else:
        quarter = 'Unknown'
    return f"{year}{quarter}"

# dataRefine 함수
def dataRefine(refDf):
    tempDf = refDf[['TBL_NM', 'ITM_NM', 'PRD_DE', 'C1_NM', 'DT']].copy()
    newColName = tempDf['TBL_NM'].unique()[0]          # 'DT' 컬럼 네임 교체
    tempDf.rename(columns={'DT': newColName}, inplace=True)
    tempDf['PRD_DE'] = tempDf['PRD_DE'].apply(month_to_quarter)
    return tempDf[~tempDf['PRD_DE'].str.contains('Unknown')]
    
def mergeTimeSeriesDataFrame(refDf):
    df = initQuaterDf()
    return df.merge(refDf, on='PRD_DE', how='left') 


#################### 상권별 소규모 상가 공실률(단위 : %) ####################
# 2018
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+36+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=201801&endPrdDe=201904&orgId=408&tblId=DT_40801_N420201_01'
refinedDf_2018 = dataRefine(urlRequest(url))

# 2019
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+36+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=201901&endPrdDe=201904&orgId=408&tblId=DT_40801_N420201_02'
refinedDf_2019 = dataRefine(urlRequest(url))

# 2020
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+36+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202001&endPrdDe=202004&orgId=408&tblId=DT_40801_N420201_03'
refinedDf_2020 = dataRefine(urlRequest(url))

# 2021
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=600+611+626+627+628+629+630+631+636+641+642+643+644+645+646+647+648+650+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202101&endPrdDe=202104&orgId=408&tblId=DT_40801_N420201_04'
refinedDf_2021 = dataRefine(urlRequest(url))

# 2022
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=600+611+626+627+628+629+630+631+636+641+642+643+644+645+646+647+648+650+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202201&endPrdDe=202204&orgId=408&tblId=DT_40801_N420201_05'
refinedDf_2022 = dataRefine(urlRequest(url))

refinedDf = pd.concat([refinedDf_2018, refinedDf_2019, refinedDf_2020, refinedDf_2021, refinedDf_2022], ignore_index=True)
smallShopDistrictVacancyRate = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
################################################################################


#################### 상권별 오피스 공실률 (단위 : %) ####################
# 2018
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=201801&endPrdDe=201804&orgId=408&tblId=DT_40801_N120201_01'
refinedDf_2018 = dataRefine(urlRequest(url))

# 2019
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=201901&endPrdDe=201904&orgId=408&tblId=DT_40801_N120201_02'
refinedDf_2019 = dataRefine(urlRequest(url))

# 2020
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202001&endPrdDe=202004&orgId=408&tblId=DT_40801_N120201_03'
refinedDf_2020 = dataRefine(urlRequest(url))

# 2021
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=100+111+126+127+128+129+130+131+141+142+143+144+145+146+147+148+150+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202101&endPrdDe=202104&orgId=408&tblId=DT_40801_N120201_04'
refinedDf_2021 = dataRefine(urlRequest(url))

# 2022
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=100+111+126+127+128+129+130+131+141+142+143+144+145+146+147+148+150+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202201&endPrdDe=202204&orgId=408&tblId=DT_40801_N120201_05'
refinedDf_2022 = dataRefine(urlRequest(url))

refinedDf = pd.concat([refinedDf_2018, refinedDf_2019, refinedDf_2020, refinedDf_2021, refinedDf_2022], ignore_index=True)
officeShopDistrictVacancyRate = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
#########################################################################


#################### 상권별 중대형 상가 공실률(단위 : %) ####################
# 2018
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+36+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=201801&endPrdDe=201804&orgId=408&tblId=DT_40801_N220201_01'
refinedDf_2018 = dataRefine(urlRequest(url))

# 2019
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+36+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=201904&endPrdDe=201904&orgId=408&tblId=DT_40801_N220201_02'
refinedDf_2019 = dataRefine(urlRequest(url))

# 2020
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=10+11+26+27+28+29+30+31+36+41+42+43+44+45+46+47+48+50+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202001&endPrdDe=202004&orgId=408&tblId=DT_40801_N220201_03'
refinedDf_2020 = dataRefine(urlRequest(url))

# 2021
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=200+211+226+227+228+229+230+231+236+241+242+243+244+245+246+247+248+250+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202104&endPrdDe=202104&orgId=408&tblId=DT_40801_N220201_04'
refinedDf_2021 = dataRefine(urlRequest(url))

# 2022
url = f'https://kosis.kr/openapi/Param/statisticsParameterData.do?method=getList&apiKey={apiKey}=&itmId=H1+&objL1=200+211+226+227+228+229+230+231+236+241+242+243+244+245+246+247+248+250+&objL2=&objL3=&objL4=&objL5=&objL6=&objL7=&objL8=&format=json&jsonVD=Y&prdSe=Q&startPrdDe=202201&endPrdDe=202204&orgId=408&tblId=DT_40801_N220201_05'
refinedDf_2022 = dataRefine(urlRequest(url))

refinedDf = pd.concat([refinedDf_2018, refinedDf_2019, refinedDf_2020, refinedDf_2021, refinedDf_2022], ignore_index=True)
middleBigShopDistrictVacancyRate = mergeTimeSeriesDataFrame(refinedDf.iloc[:, [2, 3, 4]])
############################################################################

# Dataframe을 결합하여 상권별 임대가격지수 Dataframe을 만듦 (상권별 통합상가 임대가격지수(2024 1Q부터만 데이터가 있음)는 결합대상 제외)
shopDistrictVacancyRate_Q = smallShopDistrictVacancyRate.merge(officeShopDistrictVacancyRate, on=['PRD_DE', 'C1_NM'], how='left') \
               .merge(middleBigShopDistrictVacancyRate, on=['PRD_DE', 'C1_NM'], how='left')

# 상권별 소규모 상가 공실률 : smallShopVacancy
# 상권별 오피스 공실률 : officeShopVacancy
# 상권별 중대형 상가 공실률 : midBigShopVacancy
shopDistrictVacancyRate_Q.columns = [
    'PRD_DE', 'REGION', 
    'smallShopVacancy', 'officeShopVacancy', 'midBigShopVacancy'
]
display(shopDistrictVacancyRate_Q)

# lag 생성할 대상 컬럼
target_cols = ['smallShopVacancy', 'officeShopVacancy', 'midBigShopVacancy']
lag_n = 4  # 원하는 lag 단계 수

for col in target_cols:
    # lag를 삽입할 시작 위치 계산 (컬럼 이름이 바뀌므로 매번 재계산)
    base_idx = shopDistrictVacancyRate_Q.columns.get_loc(col) + 1
    for lag in range(1, lag_n + 1):
        lag_col_name = f'{col}_{3*lag}M'
        # REGION 기준으로 그룹별 shift
        lag_series = shopDistrictVacancyRate_Q.groupby('REGION')[col].shift(lag)
        # 중간에 lag 컬럼 삽입
        shopDistrictVacancyRate_Q.insert(loc=base_idx, column=lag_col_name, value=lag_series)
        # 다음 lag 컬럼이 바로 옆에 삽입되도록 인덱스 증가
        base_idx += 1

# 'Q4'로 끝나는 row만 필터링
df_q4 = shopDistrictVacancyRate_Q[shopDistrictVacancyRate_Q['PRD_DE'].str.endswith('Q4')].copy()
df_q4['PRD_DE'] = df_q4['PRD_DE'].str.replace('Q4', '', regex=False)
shopDistrictVacancyRate = df_q4.copy()

display(shopDistrictVacancyRate)
save_excel_and_csv(shopDistrictVacancyRate, 'kosis상가오피스공실률_지역')

,PRD_DE,REGION,smallShopVacancy,officeShopVacancy,midBigShopVacancy
0,2018Q1,전국,4.7,12.7,10.4
1,2018Q1,서울,3.7,11.9,7.7
2,2018Q1,부산,6,16,9.8
3,2018Q1,대구,5.1,11.1,12.5
4,2018Q1,인천,4.4,13.4,12.8
...,...,...,...,...,...
355,2022Q4,전북,10.4968816714,15.2539751808,17.7640310488
356,2022Q4,전남,6.7621761604,24.1158573621,11.9684232042
357,2022Q4,경북,6.2488865808,22.0877965742,19.893386352
358,2022Q4,경남,7.7068800103,17.2263960105,14.4089017424


,PRD_DE,REGION,smallShopVacancy,smallShopVacancy_3M,smallShopVacancy_6M,smallShopVacancy_9M,smallShopVacancy_12M,officeShopVacancy,officeShopVacancy_3M,officeShopVacancy_6M,officeShopVacancy_9M,officeShopVacancy_12M,midBigShopVacancy,midBigShopVacancy_3M,midBigShopVacancy_6M,midBigShopVacancy_9M,midBigShopVacancy_12M
54,2018,전국,5.3,5.6,5.2,4.7,NaN,12.4,12.7,13.2,12.7,NaN,10.8,10.6,10.7,10.4,NaN
55,2018,서울,2.4,3,3.2,3.7,NaN,11.4,11.6,12.1,11.9,NaN,7,6.9,7.4,7.7,NaN
56,2018,부산,6.3,6.6,6,6,NaN,15.2,15.6,16,16,NaN,10.3,9.6,9.2,9.8,NaN
57,2018,대구,3.3,3,4,5.1,NaN,11.2,11.3,11.5,11.1,NaN,13.7,14.3,12.7,12.5,NaN
58,2018,인천,5.1,4.5,3.6,4.4,NaN,17.3,16.5,16.6,13.4,NaN,12.9,13.8,12.9,12.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,2022,전북,10.4968816714,10.1367433247,10.0377306482,9.9728108987,11.3,15.2539751808,15.1382522953,15.6853814581,14.7279884978,14.6,17.7640310488,17.358212287,16.4818251711,15.9844597922,16.9
356,2022,전남,6.7621761604,7.7140813594,6.5272612694,5.9289920864,6.1,24.1158573621,23.8412525661,22.895817114,23.5145708433,24.7,11.9684232042,12.2474222854,11.4188724856,10.9083605904,11.4
357,2022,경북,6.2488865808,6.6955745241,6.9852935036,6.5944933843,7.4,22.0877965742,23.4100603118,20.5396795251,21.2282836697,21,19.893386352,19.0765951808,18.6946109931,18.4395699398,18.5
358,2022,경남,7.7068800103,6.8388487547,6.7302949946,6.125595862,6.5,17.2263960105,18.5041846266,17.0847612352,17.2233939913,17.1,14.4089017424,14.8374203122,15.1295002628,14.8174652459,16
